@siva los cambios de la V.2.1 fueron: 
-Se elimina el filtrado que se quedó en el Timestamp Eliminator.
-Se actualizan los códigos de pesajes para que trabajen con el formato actual (07Oct24) de los archivos
-Se actualizan los códigos de comparación estadística para que trabajen con una sola fecha para futuras medicioens

# Índice de Entradas

1. [Timestamp Eliminator](#1)
2. [Curl File Generator](#2)
3. [Moduli Curl File Generator](#3)
4. [General Velocity File Generator](#4)
5. [Velocity Generator by Triad](#5)
6. [Vorticity File Generator](#6)
7. [Helicity Generator by Triads](#7)
8. [First Integral of Helicity Generator](#8)
9. [First Integral of Absolute Values of Helicity Generator](#9)
10. [General Enstrophy File Generator](#10)
11. [First Integral of Enstrophy](#11)
12. [First Integral Moduli Baseline vs Experimental Color](#12)
13. [Second Integral of Absolute Helicity Baseline vs Experimental Color](#13)


Este primer código se encarga de determinar la ruta en la que se encuentran los archivos que se van a trabajar

In [12]:
root_folder = r"D:\Code\Hypatia\Fechas\2025\25Ene25"

In [ ]:
import os
import re

def buscar_labs_en_carpetas(root_folder):
    # Expresión regular para encontrar laboratorios en formato "Lab_algo" o "LabAlgo",
    # y asegurar que la carpeta comience con un número
    patron = re.compile(r"^\d.*(Lab(?:_|)\w+)$")
    labs_encontrados = []

    for carpeta in os.listdir(root_folder):
        ruta_completa = os.path.join(root_folder, carpeta)
        if os.path.isdir(ruta_completa):
            # Buscar coincidencia con la expresión regular
            coincidencia = patron.search(carpeta)
            if coincidencia:
                # Guardar el nombre del laboratorio tal como se encuentra
                labs_encontrados.append(coincidencia.group(1))

    return labs_encontrados

labs = buscar_labs_en_carpetas(root_folder)
print("Laboratorios encontrados:", labs)

if len(labs) > 1:
    chooser = True
else:
    chooser = False

triad_names = {
    "FRT": "Frontal-Derecho-Superior",  
    "PLB": "Trasero-Izquierdo-Inferior",  
    "FLT": "Frontal-Izquierdo-Superior",  
    "PRB": "Trasero-Derecho-Inferior",  
    "FRB": "Frontal-Derecho-Inferior",  
    "PLT": "Trasero-Izquierdo-Superior",  
    "FLB": "Frontal-Izquierdo-Inferior",  
    "PRT": "Trasero-Derecho-Superior",  
    "RTB": "Derecho-Superior-Inferior",  
    "FLP": "Frontal-Izquierdo-Trasero",  
    "LTB": "Izquierdo-Superior-Inferior",  
    "FRP": "Frontal-Derecho-Trasero"
}

triad_codes = {
    (0, 2, 4): "FRT",  
    (1, 3, 5): "PLB",  
    (0, 3, 4): "FLT",  
    (1, 2, 5): "PRB",  
    (0, 2, 5): "FRB",  
    (1, 3, 4): "PLT",  
    (0, 3, 5): "FLB",  
    (1, 2, 4): "PRT",  
    (2, 4, 5): "RTB",  
    (0, 1, 3): "FLP",  
    (3, 4, 5): "LTB",  
    (0, 1, 2): "FRP"
}

# Define the triads
triads = [
    (0, 2, 4),  # T1 (FRONTAL, DERECHA, SUPERIOR)
    (1, 3, 5),  # T2 (TRASERO, IZQUIERDA, INFERIOR)
    (0, 3, 4),  # T3 (FRONTAL, IZQUIERDA, SUPERIOR)
    (1, 2, 5),  # T4 (TRASERO, DERECHA, INFERIOR)
    (0, 2, 5),  # T5 (FRONTAL, DERECHA, INFERIOR)
    (1, 3, 4),  # T6 (TRASERO, IZQUIERDA, SUPERIOR)
    (0, 3, 5),  # T7 (FRONTAL, IZQUIERDA, INFERIOR)
    (1, 2, 4),  # T8 (TRASERO, DERECHA, SUPERIOR)
    (2, 4, 5),  # T9 (DERECHA, SUPERIOR, INFERIOR)
    (0, 1, 3),  # T10 (IZQUIERDA, FRONTAL, TRASERO)
    (3, 4, 5),  # T11 (IZQUIERDA, SUPERIOR, INFERIOR)
    (0, 1, 2)   # T12 (DERECHA, FRONTAL, TRASERO)
]

file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednaranja.txt"]

color_map = {
    "amarilla": "yellow",
    "roja": "red",
    "azul": "blue",
    "verde": "green",
    "morada": "purple",
    "negra": "black",
    "naranja": "orange"
}

colores = ['Amarilla', 'Roja', 'Verde', 'Morada', 'Azul', 'Negra', 'Naranja']

## 1

$$

    \Huge \text{Second Timestamp Eliminator}

$$


In [ ]:
import os
import re

# Función para actualizar el formato de las líneas
def update_file_format(filepath):
    updated_lines = []
    print(f"Actualizando formato del archivo: {filepath}")
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            #print(f"Procesando línea: {line.strip()}")  # Mostrar cada línea procesada
            # Verificar si la línea contiene dos estampas de tiempo
            if re.match(r'^\d{2}:\d{2}:\d{2}\.\d{3} -> \d{2}:\d{2}:\d{2}\.\d{3} -> ', line):
                # Extraer y conservar sólo desde la segunda estampa de tiempo hasta el final de la línea
                new_line = re.sub(r'^\d{2}:\d{2}:\d{2}\.\d{3} -> ', '', line.strip())
                #print(f"Formato actualizado: {new_line}")  # Mostrar la línea actualizada
                updated_lines.append(new_line)
            else:
                updated_lines.append(line.strip())
    
    # Sobrescribir el archivo con el nuevo formato
    with open(filepath, 'w', encoding='utf-8') as file:
        for line in updated_lines:
            file.write(line + '\n')
    print(f"Formato actualizado en: {filepath}")

# Función para escanear un directorio y procesar archivos con sufijos específicos
def scan_directory(root_folder, file_suffixes):
    for root, dirs, files in os.walk(root_folder):
        dirs[:] = [d for d in dirs if d not in ["Data Analysis", "Programas"]]  # Excluir directorios
        for filename in files:
            if any(filename.endswith(suffix) for suffix in file_suffixes):
                file_path = os.path.join(root, filename)
                update_file_format(file_path)  # Actualizar formato del archivo

# Realiza la búsqueda y procesa los archivos
scan_directory(root_folder, file_suffixes)

## 2

$$

    \Huge \text{Curl File Generator}

$$



In [ ]:
import numpy as np
import os
from datetime import datetime, timedelta
import re

def read_sensor_data(filepath):
    """
    Lee los datos del sensor desde un archivo.
    """
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return [], np.array([])

def calculate_curl(data):
    """
    Calcula la rotacional de los datos proporcionados.
    """
    try:
        Dx = Dy = Dz = 1.0
        curl_x = (data[:, 1] - data[:, 0]) / Dx  # (Trasero - Frontal) / Dx
        curl_y = (data[:, 3] - data[:, 2]) / Dy  # (Derecha - Izquierda) / Dy
        curl_z = (data[:, 5] - data[:, 4]) / Dz  # (Inferior - Superior) / Dz
        return np.array([curl_x, curl_y, curl_z]).T
    except Exception as e:
        print(f"Error al calcular la rotacional: {e}")
        return np.array([])

def generate_new_timestamps(timestamps):
    """
    Genera nuevos timestamps comenzando desde 00:00:00.000.
    """
    base_time = datetime.strptime("00:00:00.000", '%H:%M:%S.%f')
    new_timestamps = []
    initial_time = datetime.strptime(timestamps[0], '%H:%M:%S.%f')
    
    for ts in timestamps:
        current_time = datetime.strptime(ts, '%H:%M:%S.%f')
        delta = current_time - initial_time
        new_time = base_time + delta
        new_timestamps.append(new_time.strftime('%H:%M:%S.%f')[:-3])  # Mantener el formato exacto
    return new_timestamps

def find_experiment_folder(root_folder, lab, shift):
    """
    Encuentra la carpeta del experimento según el nombre del laboratorio y el turno.
    """
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')  # Regex para detectar fechas
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder)
    return None

def process_files(input_folder, output_folder, file_suffixes=None):
    """
    Procesa los archivos dentro de la carpeta de entrada y genera archivos con los cálculos de rotacional.
    """
    generated_files = []
    for root, dirs, files in os.walk(input_folder):
        # Excluir carpetas no deseadas
        dirs[:] = [d for d in dirs if "Data Analysis" not in os.path.join(root, d)]
        
        for filename in files:
            if "00" in filename:  # Saltar archivos con "00" en el nombre
                continue
            if file_suffixes and not any(filename.endswith(suffix) for suffix in file_suffixes):
                continue
                
            file_path = os.path.join(root, filename)
            try:
                # Leer datos del archivo
                timestamps, data = read_sensor_data(file_path)
                if data.size == 0:
                    continue
                # Calcular rotacional
                curls = calculate_curl(data)
                if curls.size == 0:
                    continue
                
                # Generar nuevos timestamps comenzando desde 00:00:00.000
                adjusted_timestamps = generate_new_timestamps(timestamps)

                # Generar el nombre del archivo de salida
                output_filename = f"curl_{filename}"
                output_file_path = os.path.join(output_folder, output_filename)

                # Guardar el archivo con los nuevos datos
                with open(output_file_path, "w") as f:
                    for i, curl in enumerate(curls):
                        f.write(f"{adjusted_timestamps[i]} -> ({curl[0]:.10f}, {curl[1]:.10f}, {curl[2]:.10f})\n")
                generated_files.append(output_filename)
            except Exception as e:
                print(f"Error al procesar el archivo {file_path}: {e}")
                continue

    return generated_files

def process_lab(root_folder, lab_name):
    """
    Procesa los archivos de un laboratorio específico.
    """
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró la carpeta de experimentos para {lab_name}, turno {shift_name}.")
            continue
        
        print(f"Procesando directorio: {experiment_folder}")

        # Procesar la línea base
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Curl_Baseline")
        os.makedirs(output_folder, exist_ok=True)
        file_suffixes_baseline = ["negra.txt"]  # Procesar solo archivos "*negra.txt"
        generated_files = process_files(experiment_folder, output_folder, file_suffixes_baseline)
        
        print(f"Archivos generados para {lab_name}, turno {shift_name} Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Curl_Experimental_Color")
        os.makedirs(output_folder, exist_ok=True)
        filtered_suffixes = [suffix for suffix in file_suffixes if suffix != "mednegra.txt"]
        generated_files = process_files(experiment_folder, output_folder, filtered_suffixes)
        
        print(f"Archivos generados para {lab_name}, turno {shift_name} Experimental:")
        for file in generated_files:
            print(f"  - {file}")

# Procesar cada laboratorio
for lab_name in labs:
    process_lab(root_folder, lab_name)

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import re
from ripser import ripser
from persim import plot_diagrams
from textwrap import wrap



# Asignación de colores según sufijo
color_map = {
    "medroja": "red",
    "medmorada": "purple",
    "medazul": "blue",
    "medverde": "green",
    "medamarilla": "yellow",
    "mednaranja": "orange",
    "mednegra": "black"
}

def read_sensor_data(filepath):
    """
    Lee los datos del sensor desde un archivo.
    """
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1 and len(parts[1].strip('()').split(', ')) > 0:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
        # Validar dimensiones consistentes
        if len(timestamps) != len(data):
            min_length = min(len(timestamps), len(data))
            timestamps = timestamps[:min_length]
            data = data[:min_length]
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return [], np.array([])

def assign_color(filename, color_map):
    """
    Asigna el color basado en el sufijo del archivo.
    """
    import re  # Importar aquí en caso de problemas contextuales
    try:
        match = re.search(r"med[a-z]+", filename)  # Busca el sufijo 'med{color}'
        if match:
            suffix = match.group()  # Extrae el sufijo completo como 'mednegra', 'medroja', etc.
            if suffix in color_map:
                return color_map[suffix]  # Devuelve el color correspondiente
            else:
                print(f"Sufijo no reconocido en el archivo: {filename}")
                return "black"
        else:
            print(f"No se pudo extraer el sufijo del archivo: {filename}")
            return "black"
    except Exception as e:
        print(f"Error asignando color para el archivo {filename}: {e}")
        return "black"

def detect_abrupt_changes(curls):
    derivatives = np.diff(curls, axis=0)
    max_derivative = np.max(np.abs(derivatives), axis=0)
    num_peaks = np.sum(np.any(np.abs(derivatives) > 0.1 * np.ptp(curls, axis=0), axis=1))
    return max_derivative, num_peaks

def calculate_distances(curls_baseline, curls_mei):
    distances = np.linalg.norm(curls_mei[:, None, :] - curls_baseline[None, :, :], axis=2)
    avg_distance = np.mean(distances)
    max_distance = np.max(distances)
    std_distance = np.std(distances)
    return avg_distance, max_distance, std_distance

def calculate_trajectory_similarity(curls_baseline, curls_mei):
    baseline_norm = np.linalg.norm(curls_baseline, axis=1, keepdims=True)
    mei_norm = np.linalg.norm(curls_mei, axis=1, keepdims=True)
    baseline_normed = curls_baseline / baseline_norm
    mei_normed = curls_mei / mei_norm
    similarities = np.sum(baseline_normed[:, None, :] * mei_normed[None, :, :], axis=2)
    avg_similarity = np.mean(similarities)
    return avg_similarity

def visualize_time_series(curls_before, curls_mei, curls_after, timestamps_before, timestamps_mei, timestamps_after, lab_name, shift_name, output_folder, pair_name, mei_color):
    """
    Genera gráficos de series de tiempo para comparar la línea base antes de la MEI, la MEI y la línea base después, 
    superpuestas en el mismo rango temporal.
    """
    # Asegurar que los datos son bidimensionales, si no, crear arrays vacíos con la forma correcta
    curls_before = curls_before if curls_before.ndim == 2 else np.zeros((0, 3))
    curls_mei = curls_mei if curls_mei.ndim == 2 else np.zeros((0, 3))
    curls_after = curls_after if curls_after.ndim == 2 else np.zeros((0, 3))

    # Ajustar dimensiones si no coinciden
    min_length_before = min(len(timestamps_before), len(curls_before))
    min_length_mei = min(len(timestamps_mei), len(curls_mei))
    min_length_after = min(len(timestamps_after), len(curls_after))

    timestamps_before = timestamps_before[:min_length_before]
    curls_before = curls_before[:min_length_before]

    timestamps_mei = timestamps_mei[:min_length_mei]
    curls_mei = curls_mei[:min_length_mei]

    timestamps_after = timestamps_after[:min_length_after]
    curls_after = curls_after[:min_length_after]

    # Usar el rango temporal más amplio para alinear las series
    time_indices = np.arange(max(len(timestamps_before), len(timestamps_mei), len(timestamps_after)))

    # Componentes del rotacional
    components = ['curl_x', 'curl_y', 'curl_z']

    # Crear figura y ejes
    fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

    for i, component in enumerate(components):
        # Superponer las tres series de tiempo
        axes[i].plot(
            time_indices[:len(curls_before)], curls_before[:, i], color="black", label=f'Línea Base Antes ({component})', alpha=0.7
        )
        axes[i].plot(
            time_indices[:len(curls_mei)], curls_mei[:, i], color=mei_color, label=f'MEI ({component})', alpha=0.7
        )
        axes[i].plot(
            time_indices[:len(curls_after)], curls_after[:, i], color="gray", label=f'Línea Base Después ({component})', alpha=0.7
        )

        # Configuración de ejes
        axes[i].set_ylabel(f"{component}")
        axes[i].legend(loc='upper right')

    # Configuración del eje x
    axes[-1].set_xlabel("Índice Temporal")

    # Títulos y diseño
    fig.suptitle(f"Comparativa de Series de Tiempo - {lab_name} ({shift_name})")
    plt.tight_layout()

    # Guardar la figura
    output_path = os.path.join(output_folder, f"{pair_name}_time_series_full_comparison.png")
    plt.savefig(output_path)
    plt.close()


def detect_abrupt_changes(curls):
    if curls.size == 0:
        print("Advertencia: Datos vacíos proporcionados a detect_abrupt_changes.")
        return np.array([]), 0
    derivatives = np.diff(curls, axis=0)
    max_derivative = np.max(np.abs(derivatives), axis=0)
    num_peaks = np.sum(np.any(np.abs(derivatives) > 0.1 * np.ptp(curls, axis=0), axis=1))
    return max_derivative, num_peaks

def calculate_persistent_homology(curls, output_folder, pair_name, description):
    """
    Calcula la homología persistente de los datos y genera un diagrama de persistencia.

    Parámetros:
    - curls: Datos del espacio de fases para análisis.
    - output_folder: Carpeta donde se guardará el diagrama de persistencia.
    - pair_name: Nombre del par de comparación.
    - description: Descripción del conjunto analizado (e.g., "Baseline", "MEI").
    """
    try:
        diagrams = ripser(curls)['dgms']
        h0_count = len(diagrams[0])  # Número de componentes conexas
        h1_count = len(diagrams[1])  # Número de ciclos
        return diagrams, h0_count, h1_count

    except Exception as e:
        print(f"Error al calcular homología persistente para {pair_name} ({description}): {e}")
        return None, 0, 0

def calculate_comparative_persistence(curls_baseline, curls_mei, output_folder, pair_name):
    """
    Genera diagramas comparativos de persistencia entre Línea Base y MEI.

    Parámetros:
    - curls_baseline: Datos de la Línea Base.
    - curls_mei: Datos de la Medición Experimental.
    - output_folder: Carpeta donde se guardará el diagrama de comparación.
    - pair_name: Nombre del par de comparación.
    """
    try:
        # Calcular homología persistente para cada conjunto
        diagrams_baseline, h0_baseline, h1_baseline = calculate_persistent_homology(curls_baseline, output_folder, pair_name, "Baseline")
        diagrams_mei, h0_mei, h1_mei = calculate_persistent_homology(curls_mei, output_folder, pair_name, "MEI")

        if diagrams_baseline is None or diagrams_mei is None:
            return 0, 0, 0, 0

        # Crear el grid comparativo
        fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True, sharex=True)

        # Graficar diagramas de persistencia
        plot_diagrams(diagrams_baseline, ax=axes[0], show=False)
        axes[0].set_title("Baseline")
        
        plot_diagrams(diagrams_mei, ax=axes[1], show=False)
        axes[1].set_title("MEI")

        # Obtener límites comunes para ambos ejes
        x_min = min(axes[0].get_xlim()[0], axes[1].get_xlim()[0])
        x_max = max(axes[0].get_xlim()[1], axes[1].get_xlim()[1])
        y_min = min(axes[0].get_ylim()[0], axes[1].get_ylim()[0])
        y_max = max(axes[0].get_ylim()[1], axes[1].get_ylim()[1])

        # Ajustar límites en ambos ejes
        for ax in axes:
            ax.set_xlim(x_min, x_max)
            ax.set_ylim(y_min, y_max)

        # Configuración y guardado
        fig.suptitle(f"Comparación de Diagramas de Persistencia ({pair_name})")
        comparison_path = os.path.join(output_folder, f"{pair_name}_Comparative_Persistence.png")
        plt.savefig(comparison_path, bbox_inches='tight')
        plt.close()

        return h0_baseline, h1_baseline, h0_mei, h1_mei

    except Exception as e:
        print(f"Error al calcular homología persistente comparativa para {pair_name}: {e}")
        return 0, 0, 0, 0


def hierarchical_analysis(curls_baseline, curls_mei, timestamps, lab_name, shift_name, output_folder, pair_name, baseline_color, mei_color, curls_before=None, curls_after=None, timestamps_before=None, timestamps_after=None):
    """
    Realiza un análisis jerárquico y escribe la conclusión directamente en el archivo de resultados.
    """
    if curls_baseline.size == 0 or curls_mei.size == 0:
        print(f"Advertencia: Datos vacíos detectados en {pair_name}. Saltando análisis.")
        return

    result_path = os.path.join(output_folder, f"{pair_name}_Results.txt")
    conclusion = "Prueba no concluyente, realizar más análisis"  # Conclusión predeterminada

    with open(result_path, "w") as f:

        # Nivel 4: Histograma Temporalizados
        f.write("Nivel 1: Analisis de Series de Tiempo.\n")
        if curls_before is not None and curls_after is not None:
            visualize_time_series(curls_before, curls_mei, curls_after, timestamps_before, timestamps, timestamps_after, lab_name, shift_name, output_folder, pair_name, mei_color)

        # Nivel 2: Cambios abruptos
        max_derivative, num_peaks = detect_abrupt_changes(curls_mei)
        f.write(f"Nivel 2: Max derivative: {max_derivative}, Num peaks: {num_peaks}\n")
        if num_peaks < 10:
            conclusion = "No hubo cambio significativo"
            f.write(f"Conclusión: {conclusion}\n")
            return
        
        # Nivel 3: Distancias temporalizadas
        avg_distance, max_distance, std_distance = calculate_distances(curls_baseline, curls_mei)
        f.write(f"Nivel 3: Avg distance: {avg_distance}, Max distance: {max_distance}, Std distance: {std_distance}\n")
        if avg_distance < 0.05:
            conclusion = "No hubo cambio significativo"
            f.write(f"Conclusión: {conclusion}\n")
            return
        
        # Nivel 4: Similitud de trayectorias
        avg_similarity = calculate_trajectory_similarity(curls_baseline, curls_mei)
        f.write(f"Nivel 4: Avg trajectory similarity: {avg_similarity}\n")
        if avg_similarity > 0.9:
            conclusion = "No hubo cambio significativo"
            f.write(f"Conclusión: {conclusion}\n")
            return
        
        # Nivel 5: Homología persistente
        f.write("Nivel 5: Homología persistente.\n")
        h0_baseline, h1_baseline, h0_mei, h1_mei = calculate_comparative_persistence(curls_baseline, curls_mei, output_folder, pair_name)

        # Guardar los resultados en el archivo
        f.write(f"Nivel 5: H0 (LB - componentes conexas): {h0_baseline}, H1 (LB - ciclos): {h1_baseline}\n")
        f.write(f"Nivel 5: H0 (MEI - componentes conexas): {h0_mei}, H1 (MEI - ciclos): {h1_mei}\n")

        # Determinar conclusión final
        if h1_mei > 1500:
            conclusion = "Si hubo un cambio significativo"
        elif h1_mei > 1000:
            conclusion = "Hubo un cambio significativo ligero"
        elif h1_mei <= 1000:
            conclusion = "No hubo cambio significativo"

        # Escribir conclusión al archivo
        f.write(f"Conclusión: {conclusion}\n")

        
def consolidate_results(root_folder, labs):
    """
    Barre todos los resultados generados en los laboratorios y consolida en un archivo único.
    """
    consolidated_file = os.path.join(root_folder, "Consolidated_Results.txt")
    
    with open(consolidated_file, "w") as output_file:
        for lab_name in labs:
            output_file.write(f"Laboratorio: {lab_name}\n")
            output_file.write("=" * 50 + "\n")
            
            for shift in ['V', 'M']:
                shift_name = "Vespertino" if shift == 'V' else "Matutino"
                experiment_folder = find_experiment_folder(root_folder, lab_name, shift)
                
                if not experiment_folder:
                    output_file.write(f"No se encontraron experimentos para {lab_name}, turno {shift_name}.\n")
                    continue
                
                results_folder = os.path.join(experiment_folder, "Data Analysis", "Topological Data Analysis")
                if not os.path.exists(results_folder):
                    output_file.write(f"No se encontró la carpeta de resultados para {lab_name}, turno {shift_name}.\n")
                    continue
                
                # Leer todos los archivos de resultados
                result_files = sorted([f for f in os.listdir(results_folder) if f.endswith("_Results.txt")])
                if not result_files:
                    output_file.write(f"No se encontraron archivos de resultados en {results_folder}.\n")
                    continue
                
                output_file.write(f"Turno: {shift_name}\n")
                output_file.write("-" * 50 + "\n")
                for result_file in result_files:
                    result_path = os.path.join(results_folder, result_file)
                    with open(result_path, "r") as rf:
                        output_file.write(f"Resultados de {result_file}:\n")
                        output_file.write(rf.read() + "\n")
                        output_file.write("-" * 50 + "\n")
            
            output_file.write("\n")
    
    print(f"Resultados consolidados en: {consolidated_file}")

def process_lab(root_folder, lab_name):
    """
    Procesa los datos de un laboratorio específico para turnos matutino y vespertino.
    Realiza análisis jerárquicos hasta el nivel requerido, generando resultados y gráficos según sea necesario.
    """
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró la carpeta de experimentos para {lab_name}, turno {shift_name}.")
            continue

        baseline_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Curl_Baseline")
        mei_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Curl_Experimental_Color")
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Topological Data Analysis")
        os.makedirs(output_folder, exist_ok=True)

        if not os.path.exists(baseline_folder) or not os.path.exists(mei_folder):
            print(f"Carpetas faltantes en {experiment_folder}.")
            continue

        baseline_files = sorted([f for f in os.listdir(baseline_folder) if "curl" in f])
        mei_files = sorted([f for f in os.listdir(mei_folder) if "curl" in f])

        # Nivel 1-5: Generar combinaciones y procesar todos los niveles para cada archivo MEI
        for i, mei_file in enumerate(mei_files):
            timestamps_mei, curls_mei = read_sensor_data(os.path.join(mei_folder, mei_file))
            mei_color = assign_color(mei_file, color_map)

            # Comparar con `Baseline n` y `Baseline n+1`
            for baseline_index in [i, i + 1]:
                if baseline_index >= len(baseline_files):
                    continue

                baseline_file = baseline_files[baseline_index]
                timestamps_baseline, curls_baseline = read_sensor_data(os.path.join(baseline_folder, baseline_file))
                pair_name = f"{baseline_file.split('.')[0]}_VS_{mei_file.split('.')[0]}"

                # Nivel 1: Visualización de series de tiempo
                visualize_time_series(
                    curls_baseline if baseline_index == i else np.array([]),
                    curls_mei,
                    curls_baseline if baseline_index == i + 1 else np.array([]),
                    timestamps_baseline if baseline_index == i else [],
                    timestamps_mei,
                    timestamps_baseline if baseline_index == i + 1 else [],
                    lab_name,
                    shift_name,
                    output_folder,
                    pair_name,
                    mei_color,
                )

                # Nivel 2-4: Análisis jerárquico
                hierarchical_analysis(
                    curls_baseline,
                    curls_mei,
                    timestamps_mei,
                    lab_name,
                    shift_name,
                    output_folder,
                    pair_name,
                    "black",
                    mei_color,
                )

                # Nivel 5: Análisis topológico
                diagrams_baseline, h0_baseline, h1_baseline = calculate_persistent_homology(
                    curls_baseline, output_folder, pair_name, "Baseline"
                )
                diagrams_mei, h0_mei, h1_mei = calculate_persistent_homology(
                    curls_mei, output_folder, pair_name, "MEI"
                )
                calculate_comparative_persistence(curls_baseline, curls_mei, output_folder, pair_name)




# Procesa cada laboratorio
for lab_name in labs:
    process_lab(root_folder, lab_name)


# Consolidar resultados
consolidate_results(root_folder, labs)

In [ ]:
import os
import numpy as np
from ripser import ripser
import pandas as pd
import matplotlib.pyplot as plt
from gudhi.wasserstein import wasserstein_distance
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform


def read_sensor_data(filepath):
    """
    Lee los datos del sensor desde un archivo, manejando posibles errores de lectura.
    """
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
    except Exception as e:
        print(f"Error al leer {filepath}: {e}")
    return timestamps, np.array(data)

def calculate_geometric_dispersion(curls):
    """
    Calcula métricas de dispersión geométrica a partir de los datos.
    """
    distances = pdist(curls)
    mean_distance = np.mean(distances)
    max_distance = np.max(distances)
    return {
        "Mean_Distance": mean_distance,
        "Max_Distance": max_distance
    }

def calculate_trajectory_curvature(curls):
    """
    Calcula la curvatura promedio de las trayectorias.
    """
    deltas = np.diff(curls, axis=0)
    norms = np.linalg.norm(deltas, axis=1)
    curvatures = np.abs(np.diff(norms)) / (norms[:-1] + 1e-8)
    return {
        "Average_Curvature": np.mean(curvatures),
        "Max_Curvature": np.max(curvatures)
    }



def calculate_density_metrics(curls, eps=0.5, min_samples=5):
    """
    Calcula métricas de densidad usando DBSCAN.
    """
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(curls)
    num_clusters = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
    noise_points = np.sum(clustering.labels_ == -1)
    return {
        "Num_Clusters": num_clusters,
        "Noise_Points": noise_points
    }


def calculate_fractal_dimension(data, epsilon_range):
    """
    Calcula la dimensión fractal de los datos mediante el método de conteo de cajas (box-counting).
    """
    from scipy.spatial.distance import pdist, squareform
    dists = squareform(pdist(data))
    counts = []
    for epsilon in epsilon_range:
        counts.append(np.sum(dists < epsilon))
    coeffs = np.polyfit(np.log(epsilon_range), np.log(counts), 1)
    return -coeffs[0]

def calculate_persistence_entropy(diagram):
    """
    Calcula la entropía de persistencia a partir de un diagrama de persistencia.
    """
    lifetimes = [d[1] - d[0] for d in diagram if d[1] < np.inf]
    if len(lifetimes) > 0:
        probs = np.array(lifetimes) / sum(lifetimes)
        return entropy(probs)
    return 0


def calculate_average_curvature(curls):
    """
    Calcula la curvatura promedio de las trayectorias.
    """
    deltas = np.diff(curls, axis=0)
    norms = np.linalg.norm(deltas, axis=1)
    curvatures = np.abs(np.diff(norms)) / (norms[:-1] + 1e-8)
    return np.mean(curvatures)

def calculate_local_density(data, eps=0.5, min_samples=5):
    """
    Calcula la densidad local de puntos en el espacio usando DBSCAN.
    """
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(data)
    return len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)


def generate_persistence_diagram(diagrams, output_folder, filename):
    """
    Genera y guarda un diagrama de persistencia en un archivo de imagen.
    
    Parameters:
        diagrams (list): Diagramas de persistencia generados por Ripser.
        output_folder (str): Carpeta donde se guardará la imagen.
        filename (str): Nombre del archivo para el diagrama.
    """
    plt.figure(figsize=(8, 6))
    for dim, diagram in enumerate(diagrams):
        plt.scatter(
            [p[0] for p in diagram if p[1] < np.inf],
            [p[1] for p in diagram if p[1] < np.inf],
            label=f"H{dim}"
        )
    plt.xlabel("Birth")
    plt.ylabel("Death")
    plt.title(f"Diagrama de Persistencia: {filename}")
    plt.legend()
    plt.grid(True)

    # Crear la carpeta si no existe
    os.makedirs(output_folder, exist_ok=True)
    diagram_path = os.path.join(output_folder, f"{filename}_persistence_diagram.png")
    plt.savefig(diagram_path)
    plt.close()
    print(f"Diagrama de persistencia guardado en: {diagram_path}")


def calculate_homology(curls):
    """
    Calcula la homología persistente de los datos transformados exponencialmente.
    """
    try:
        transformed_curls = transform_exponential_abs(curls)
        results = ripser(transformed_curls, maxdim=1, thresh=0.5)
        diagrams = results.get('dgms', [])
        
        if not diagrams or len(diagrams[0]) == 0:
            print("Diagramas de persistencia vacíos después de la transformación exponencial con valor absoluto.")
            return {
                "H0_count": 0,
                "H1_count": 0,
                "Max_Persistence_H0": 0,
                "Max_Persistence_H1": 0,
                "Diagrams": diagrams
            }

        # Calcular métricas
        h0 = len(diagrams[0])
        h1 = len(diagrams[1]) if len(diagrams) > 1 else 0
        max_persistence_h0 = max([d[1] - d[0] for d in diagrams[0] if d[1] < np.inf], default=0)
        max_persistence_h1 = max([d[1] - d[0] for d in diagrams[1] if d[1] < np.inf], default=0)

        return {
            "H0_count": h0,
            "H1_count": h1,
            "Max_Persistence_H0": max_persistence_h0,
            "Max_Persistence_H1": max_persistence_h1,
            "Diagrams": diagrams
        }
    except Exception as e:
        print(f"Error al calcular homología: {e}")
        return {
            "H0_count": 0,
            "H1_count": 0,
            "Max_Persistence_H0": 0,
            "Max_Persistence_H1": 0,
            "Diagrams": []
        }



def transform_exponential_abs(curls):
    """
    Aplica una transformación exponencial a cada entrada de curls.
    """
    return np.exp(np.abs(curls))


def calculate_wasserstein_distance(diagram1, diagram2):
    """
    Calcula la distancia Wasserstein entre dos diagramas de persistencia.
    """
    return wasserstein_distance(diagram1, diagram2)




def process_baseline_file(filepath):
    """
    Procesa un archivo de línea base para calcular métricas clave y homología persistente.
    """
    timestamps, curls = read_sensor_data(filepath)
    if curls.size == 0:
        return None
    
    # Derivadas y picos
    derivatives = np.abs(np.diff(curls, axis=0))
    max_derivative = np.max(derivatives, axis=0)
    median_derivative = np.median(derivatives, axis=0)
    num_peaks = np.sum(derivatives > 0.1 * np.ptp(curls, axis=0), axis=0)

    # Homología Persistente
    homology_metrics = calculate_homology(curls)

    diagrams = homology_metrics.pop("Diagrams", [])

    # Generar y guardar diagrama de persistencia
    output_folder = os.path.join(os.path.dirname(filepath), "Persistence_Diagrams")
    filename = os.path.splitext(os.path.basename(filepath))[0]
    generate_persistence_diagram(diagrams, output_folder, filename)

    # Métricas avanzadas
    epsilon_range = np.logspace(-2, 0, 10)
    fractal_dimension = calculate_fractal_dimension(curls, epsilon_range)

    # Validar y calcular entropía de persistencia
    if 'dgms' in homology_metrics:
        diagrams = homology_metrics['dgms']
        persistence_entropy_h0 = calculate_persistence_entropy(diagrams[0]) if len(diagrams) > 0 else 0
        persistence_entropy_h1 = calculate_persistence_entropy(diagrams[1]) if len(diagrams) > 1 else 0
    else:
        persistence_entropy_h0 = 0
        persistence_entropy_h1 = 0

    # Calcular otras métricas avanzadas
    average_curvature = calculate_average_curvature(curls)
    geometric_dispersion = calculate_geometric_dispersion(curls)
    curvature_metrics = calculate_trajectory_curvature(curls)
    density_metrics = calculate_density_metrics(curls)

    # Consolidar métricas utilizando `update`
    metrics = {
        "Max_Derivative_X": max_derivative[0],
        "Max_Derivative_Y": max_derivative[1],
        "Max_Derivative_Z": max_derivative[2],
        "Median_Derivative_X": median_derivative[0],
        "Median_Derivative_Y": median_derivative[1],
        "Median_Derivative_Z": median_derivative[2],
        "Num_Peaks_X": num_peaks[0],
        "Num_Peaks_Y": num_peaks[1],
        "Num_Peaks_Z": num_peaks[2],
        "Fractal_Dimension": fractal_dimension,
        "Persistence_Entropy_H0": persistence_entropy_h0,
        "Persistence_Entropy_H1": persistence_entropy_h1,
        "Average_Curvature": average_curvature
    }
    # Agregar los diccionarios adicionales
    metrics.update(homology_metrics)
    metrics.update(geometric_dispersion)
    metrics.update(curvature_metrics)
    metrics.update(density_metrics)

    return metrics

def process_mei_file(filepath):
    """
    Procesa un archivos MEI para calcular métricas clave y homología persistente.
    """
    timestamps, curls = read_sensor_data(filepath)
    if curls.size == 0:
        return None
    
    # Derivadas y picos
    derivatives = np.abs(np.diff(curls, axis=0))
    max_derivative = np.max(derivatives, axis=0)
    median_derivative = np.median(derivatives, axis=0)
    num_peaks = np.sum(derivatives > 0.1 * np.ptp(curls, axis=0), axis=0)

    # Homología Persistente
    homology_metrics = calculate_homology(curls)

    diagrams = homology_metrics.pop("Diagrams", [])

    # Generar y guardar diagrama de persistencia
    output_folder = os.path.join(os.path.dirname(filepath), "Persistence_Diagrams")
    filename = os.path.splitext(os.path.basename(filepath))[0]
    generate_persistence_diagram(diagrams, output_folder, filename)

    # Métricas avanzadas
    epsilon_range = np.logspace(-2, 0, 10)
    fractal_dimension = calculate_fractal_dimension(curls, epsilon_range)

    # Validar y calcular entropía de persistencia
    if 'dgms' in homology_metrics:
        diagrams = homology_metrics['dgms']
        persistence_entropy_h0 = calculate_persistence_entropy(diagrams[0]) if len(diagrams) > 0 else 0
        persistence_entropy_h1 = calculate_persistence_entropy(diagrams[1]) if len(diagrams) > 1 else 0
    else:
        persistence_entropy_h0 = 0
        persistence_entropy_h1 = 0

    # Calcular otras métricas avanzadas
    average_curvature = calculate_average_curvature(curls)
    geometric_dispersion = calculate_geometric_dispersion(curls)
    curvature_metrics = calculate_trajectory_curvature(curls)
    density_metrics = calculate_density_metrics(curls)

    # Consolidar métricas utilizando `update`
    metrics = {
        "Max_Derivative_X": max_derivative[0],
        "Max_Derivative_Y": max_derivative[1],
        "Max_Derivative_Z": max_derivative[2],
        "Median_Derivative_X": median_derivative[0],
        "Median_Derivative_Y": median_derivative[1],
        "Median_Derivative_Z": median_derivative[2],
        "Num_Peaks_X": num_peaks[0],
        "Num_Peaks_Y": num_peaks[1],
        "Num_Peaks_Z": num_peaks[2],
        "Fractal_Dimension": fractal_dimension,
        "Persistence_Entropy_H0": persistence_entropy_h0,
        "Persistence_Entropy_H1": persistence_entropy_h1,
        "Average_Curvature": average_curvature
    }
    # Agregar los diccionarios adicionales
    metrics.update(homology_metrics)
    metrics.update(geometric_dispersion)
    metrics.update(curvature_metrics)
    metrics.update(density_metrics)

    return metrics


def generate_distributions(metrics_list, output_folder, lab_name, shift_name):
    """
    Genera distribuciones para las métricas por laboratorio y turno y las guarda como gráficos.
    """
    if not metrics_list:
        print(f"No hay métricas para generar distribuciones en {lab_name}, turno {shift_name}.")
        return

    df = pd.DataFrame(metrics_list)
    
    # Seleccionar solo columnas numéricas
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    distribution_folder = os.path.join(output_folder, f"Metric_Distributions_{lab_name}_{shift_name}")
    os.makedirs(distribution_folder, exist_ok=True)

    for column in numeric_columns:
        plt.figure(figsize=(8, 6))
        df[column].plot(kind='hist', bins=20, alpha=0.75, title=f"Distribución de {column} - {lab_name} ({shift_name})")
        plt.xlabel(column)
        plt.ylabel("Frecuencia")
        plt.grid(True)
        plt.savefig(os.path.join(distribution_folder, f"{column}_distribution_{lab_name}_{shift_name}.png"))
        plt.close()

    print(f"Distribuciones guardadas en: {distribution_folder}")


def calculate_local_fractal_dimension(data, epsilon_range):
    """
    Calcula la dimensión fractal local para cada punto en los datos.
    """
    from scipy.spatial.distance import pdist, squareform

    # Calculamos las distancias pairwise
    dists = squareform(pdist(data))

    dimensions = []
    for epsilon in epsilon_range:
        # Contar cuántos puntos están dentro de cada epsilon para cada punto
        counts = np.sum(dists < epsilon, axis=1)
        
        # Filtrar para asegurarnos de no tener valores vacíos o inválidos
        valid_counts = counts[counts > 1]  # Excluir casos donde el conteo es 0 o 1 (sin sentido para log)

        if len(valid_counts) == 0:  # Evitar errores si no hay valores válidos
            continue
        
        # Ajustar una línea solo si hay suficientes valores válidos
        try:
            coeffs = np.polyfit(np.log(epsilon_range[:len(valid_counts)]), np.log(valid_counts), 1)
            dimensions.append(-coeffs[0])
        except Exception as e:
            print(f"Error en ajuste de polinomio para epsilon {epsilon}: {e}")
            continue

    # Retornar la dimensión fractal promedio
    if len(dimensions) > 0:
        return np.mean(dimensions)
    else:
        print("No se pudo calcular la dimensión fractal local: todos los valores fueron inválidos.")
        return 0



def process_lab(root_folder, lab_name):
    """
    Procesa las líneas base de un laboratorio y genera tablas para cada archivo.
    También consolida los resultados en un archivo CSV único por laboratorio y turno.
    """
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró la carpeta de experimentos para {lab_name}, turno {shift_name}.")
            continue

        baseline_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Curl_Baseline")
        mei_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Curl_Experimental_Color")
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Topological Data Analysis Baseline")

        if not os.path.exists(baseline_folder):
            print(f"No se encontró la carpeta de líneas base en {baseline_folder}.")
            continue
        if not os.path.exists(mei_folder):
            print(f"No se encontró la carpeta experimental en {mei_folder}.")
            continue

        os.makedirs(output_folder, exist_ok=True)

        print(f"Procesando laboratorio: {lab_name}, turno: {shift_name}")
        baseline_files = sorted([f for f in os.listdir(baseline_folder) if "curl" in f])
        mei_files = sorted([f for f in os.listdir(mei_folder) if "curl" in f])
        all_metrics = []

        for file in baseline_files:
            filepath = os.path.join(baseline_folder, file)
            metrics = process_baseline_file(filepath)
            if metrics:
                metrics['Lab'] = lab_name
                metrics['Shift'] = shift_name
                metrics['File'] = file
                all_metrics.append(metrics)
                print(f"Métricas procesadas para {file}: {metrics}")

        for file in mei_files:
            filepath = os.path.join(mei_folder, file)
            metrics = process_mei_file(filepath)
            if metrics:
                metrics['Lab'] = lab_name
                metrics['Shift'] = shift_name
                metrics['File'] = file
                all_metrics.append(metrics)
                print(f"Métricas procesadas para {file}: {metrics}")



        # Guardar métricas consolidadas por laboratorio y turno
        consolidated_csv_path = os.path.join(root_folder, f"Consolidated_Metrics_{lab_name}_{shift_name}.csv")
        pd.DataFrame(all_metrics).to_csv(consolidated_csv_path, index=False)
        print(f"Métricas consolidadas guardadas en: {consolidated_csv_path}")

        # Generar distribuciones para las métricas por laboratorio y turno
        generate_distributions(all_metrics, root_folder, lab_name, shift_name)




# Ejecutar para todos los laboratorios
for lab in labs:
    process_lab(root_folder, lab)


## 3

$$

    \Huge \text{Moduli Curl File Generator}

$$



In [ ]:
import numpy as np
import os
import re
from datetime import datetime, timedelta

def read_sensor_data(filepath):
    """
    Lee los datos del sensor desde un archivo.
    """
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return [], np.array([])

def calculate_magnitude(data):
    """
    Calcula la magnitud de los datos proporcionados.
    """
    try:
        #print(np.linalg.norm(data, axis=1))
        return np.linalg.norm(data, axis=1)
    except Exception as e:
        print(f"Error al calcular la magnitud: {e}")
        return np.array([])

def generate_new_timestamps(timestamps):
    """
    Genera nuevos timestamps comenzando desde 00:00:00.000.
    """
    base_time = datetime.strptime("00:00:00.000", '%H:%M:%S.%f')
    new_timestamps = []
    initial_time = datetime.strptime(timestamps[0], '%H:%M:%S.%f')
    
    for ts in timestamps:
        current_time = datetime.strptime(ts, '%H:%M:%S.%f')
        delta = current_time - initial_time
        new_time = base_time + delta
        new_timestamps.append(new_time.strftime('%H:%M:%S.%f')[:-3])  # Mantener el formato exacto
    return new_timestamps

def find_experiment_folder(root_folder, lab, shift):
    """
    Encuentra la carpeta del experimento según el nombre del laboratorio y el turno.
    """
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')  # Regex para detectar fechas
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder)
    return None

def process_files(input_folder, output_folder, file_suffixes=None):
    """
    Procesa los archivos dentro de la carpeta de entrada y genera archivos con las magnitudes de rotacional.
    """
    generated_files = []
    for root, dirs, files in os.walk(input_folder):
        # Excluir carpetas no deseadas
        dirs[:] = [d for d in dirs if "Data Analysis" not in os.path.join(root, d)]
        
        for filename in files:
            if "00" in filename:  # Saltar archivos con "00" en el nombre
                continue
            if file_suffixes and not any(filename.endswith(suffix) for suffix in file_suffixes):
                continue
                
            file_path = os.path.join(root, filename)
            try:
                # Leer datos del archivo
                timestamps, data = read_sensor_data(file_path)
                if data.size == 0:
                    continue
                # Calcular magnitud de rotacional
                magnitudes = calculate_magnitude(data)
                if magnitudes.size == 0:
                    continue

                # Generar nuevos timestamps comenzando desde 00:00:00.000
                adjusted_timestamps = generate_new_timestamps(timestamps)

                # Generar el nombre del archivo de salida
                magnitude_filename = f"magnitude_{filename}"
                magnitude_file_path = os.path.join(output_folder, magnitude_filename)

                # Guardar el archivo con los nuevos datos
                with open(magnitude_file_path, "w") as f:
                    for i, magnitude in enumerate(magnitudes):
                        f.write(f"{adjusted_timestamps[i]} -> {magnitude:.10f}\n")
                generated_files.append(magnitude_filename)
            except Exception as e:
                print(f"Error al procesar el archivo {file_path}: {e}")
                continue

    return generated_files

def process_lab(root_folder, lab_name):
    """
    Procesa los archivos de un laboratorio específico para ambos turnos: Matutino y Vespertino.
    """
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró la carpeta de experimentos para {lab_name}, turno {shift_name}.")
            continue
        
        print(f"Procesando directorio: {experiment_folder}")

        # Procesar la línea base
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Moduli_Curl_Baseline")
        file_suffixes_baseline = ["negra.txt"]  # Procesar solo archivos "*negra.txt"
        os.makedirs(output_folder, exist_ok=True)
        generated_files = process_files(experiment_folder, output_folder,file_suffixes_baseline)
        
        print(f"Archivos generados para {lab_name}, turno {shift_name} Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Moduli_Curl_Experimental_Color")
        os.makedirs(output_folder, exist_ok=True)
        #file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt", "mednaranja.txt"]
        generated_files = process_files(experiment_folder, output_folder, file_suffixes)
        
        print(f"Archivos generados para {lab_name}, turno {shift_name} Experimental:")
        for file in generated_files:
            print(f"  - {file}")

# Procesar cada laboratorio
for lab_name in labs:
    process_lab(root_folder, lab_name)


## 4

$$

    \Huge \text{General Velocity File Generator}

$$


In [ ]:
import numpy as np
import os
import re
from datetime import datetime, timedelta

def read_sensor_data(filepath):
    """
    Lee los datos del sensor desde un archivo.
    """
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])  # Guardar marca de tiempo
                    numbers = parts[1].strip('()').split(', ')
                    data.append([float(num) for num in numbers])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return [], np.array([])

def calculate_velocities(data, delta_t):
    """
    Calcula las velocidades usando diferencias hacia adelante.
    """
    velocities = (data[1:] - data[:-1]) / delta_t
    return velocities

def generate_new_timestamps(timestamps):
    """
    Genera nuevos timestamps comenzando desde 00:00:00.000.
    """
    base_time = datetime.strptime("00:00:00.000", '%H:%M:%S.%f')
    new_timestamps = []
    initial_time = datetime.strptime(timestamps[0], '%H:%M:%S.%f')
    
    for ts in timestamps[:-1]:  # Excluir la última marca de tiempo
        current_time = datetime.strptime(ts, '%H:%M:%S.%f')
        delta = current_time - initial_time
        new_time = base_time + delta
        new_timestamps.append(new_time.strftime('%H:%M:%S.%f')[:-3])  # Mantener el formato exacto
    return new_timestamps

def find_experiment_folder(root_folder, lab, shift):
    """
    Encuentra la carpeta del experimento según el nombre del laboratorio y el turno.
    """
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder)
    return None

def process_files(input_folder, output_folder, delta_t, file_suffixes=None):
    """
    Procesa los archivos dentro de la carpeta de entrada y genera archivos con las velocidades calculadas.
    """
    os.makedirs(output_folder, exist_ok=True)  # Asegurar que el directorio de salida exista
    generated_files = []  # Lista para almacenar los nombres de los archivos generados

    for root, dirs, files in os.walk(input_folder):
        # Excluir carpetas que contienen 'Data Analysis'
        dirs[:] = [d for d in dirs if "Data Analysis" not in os.path.join(root, d)]
        for filename in files:
            if "00" in filename:  # Omitir archivos con "00" en el nombre
                continue
            if file_suffixes and not any(filename.endswith(suffix) for suffix in file_suffixes):
                continue

            file_path = os.path.join(root, filename)
            try:
                timestamps, data = read_sensor_data(file_path)
                if data.size == 0:
                    continue

                # Calcular velocidades
                velocities = calculate_velocities(data, delta_t)
                if velocities.size == 0:
                    continue

                # Generar nuevos timestamps comenzando desde 00:00:00.000
                adjusted_timestamps = generate_new_timestamps(timestamps)

                # Generar el nombre del archivo de salida
                output_filename = f"velocity_{filename}"
                output_file_path = os.path.join(output_folder, output_filename)

                # Guardar el archivo con los nuevos datos
                with open(output_file_path, "w") as f:
                    for i, velocity in enumerate(velocities):
                        f.write(f"{adjusted_timestamps[i]} -> ({velocity[0]:.10f}, {velocity[1]:.10f}, {velocity[2]:.10f}, {velocity[3]:.10f}, {velocity[4]:.10f}, {velocity[5]:.10f})\n")
                generated_files.append(output_filename)  # Agregar el nombre del archivo generado a la lista
            except Exception as e:
                print(f"Error al procesar el archivo {file_path}: {e}")
                continue

    return generated_files  # Devolver la lista de archivos generados

def process_lab(root_folder, lab_name, delta_t):
    """
    Procesa los archivos de un laboratorio específico para ambos turnos: Matutino y Vespertino.
    """
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró ningún directorio para {lab_name}, turno {shift_name}.")
            continue

        # Procesar la línea base
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Velocity_Baseline")
        file_suffixes_baseline = ["negra.txt"]  # Procesar solo archivos "*negra.txt"
        os.makedirs(output_folder, exist_ok=True)
        generated_files = process_files(experiment_folder, output_folder, delta_t,file_suffixes_baseline)
        
        print(f"Archivos generados para {lab_name}, turno {shift_name} Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        output_folder = os.path.join(experiment_folder, "Data Analysis", "Processing Data", "Velocity_Experimental_Color")
        os.makedirs(output_folder, exist_ok=True)
        #file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednaranja.txt"]
        generated_files = process_files(experiment_folder, output_folder, delta_t, file_suffixes)
        
        print(f"Archivos generados para {lab_name}, turno {shift_name} Experimental:")
        for file in generated_files:
            print(f"  - {file}")

# Paso de tiempo para el cálculo de velocidades
delta_t = 0.01  # Paso de tiempo (segundos) entre cada muestra

# Procesar cada laboratorio
for lab_name in labs:
    process_lab(root_folder, lab_name, delta_t)


## 5

$$

    \Huge \text{Velocity Generator by Triad}

$$


In [ ]:
#Calculo de velocidades por triada .txt
import os

def extract_velocity_components(root_folder):
    generated_files = []  # Lista para almacenar los nombres de los archivos generados
    
    for root, dirs, files in os.walk(root_folder):
        if "Velocity_Baseline" in root or "Velocity_Experimental_Color" in root:
            # Usar la carpeta actual, sin agregar '_Component'
            new_folder = root
            
            # Iterar sobre cada archivo en la carpeta original
            for filename in files:
                if filename.startswith("v") and filename.endswith(".txt"):
                    path = os.path.join(root, filename)
                    
                    with open(path, 'r') as file:
                        file_content = file.readlines()  # Leer todo el contenido una vez
                        
                        # Generar archivos por cada triada y usando su código del diccionario
                        for triad, code in triad_codes.items():
                            triad_folder = os.path.join(new_folder, code)
                            os.makedirs(triad_folder, exist_ok=True)  # Crear la carpeta de la triada si no existe
                            new_path = os.path.join(triad_folder, f"Velocity_{code}_" + filename)
                            
                            with open(new_path, 'w') as triad_file:
                                for line in file_content:
                                    time_stamp, velocities = line.strip().split(' -> ')
                                    v = eval(velocities)
                                    selected_components = tuple(v[j] for j in triad)
                                    triad_file.write(f"{time_stamp} -> {selected_components}\n")
                            
                            # Almacenar solo el nombre de la carpeta de cabecera y los nombres de los archivos generados
                            header_folder = os.path.basename(triad_folder)
                            file_name = os.path.basename(new_path)
                            generated_files.append(f"{header_folder}/{file_name}")

    return generated_files  # Devolver la lista de archivos generados

# La ruta base donde están las carpetas de origen
generated_files = extract_velocity_components(root_folder)

# Imprimir los nombres de los archivos generados
print("Generated files:")
for file in generated_files:
    print(f"  - {file}")

## 6

$$

    \Huge \text{Vorticity File Generator}

$$


In [ ]:
#Vorticidad .txt
import os
import numpy as np
import re

def read_velocity_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])  # Guardar timestamps directamente del archivo
                    velocity_components = parts[1].strip('()').split(', ')
                    if len(velocity_components) == 6:  # Asegurarse de que haya 6 componentes
                        data.append([float(v) for v in velocity_components])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return [], np.array([])

def calculate_vorticity(velocities, indices):
    if velocities.ndim != 2 or velocities.shape[1] != 3:
        print(f"velocities.ndim: {velocities.ndim}, velocities.shape: {velocities.shape}")
        raise ValueError("El array de velocidades debe ser bidimensional con tres columnas.")
    
    Vx, Vy, Vz = indices
    dvz_dy = np.gradient(velocities[:, Vz], axis=0)
    dvy_dz = np.gradient(velocities[:, Vy], axis=0)
    dvx_dz = np.gradient(velocities[:, Vx], axis=0)
    dvz_dx = np.gradient(velocities[:, Vz], axis=0)
    dvy_dx = np.gradient(velocities[:, Vy], axis=0)
    dvx_dy = np.gradient(velocities[:, Vx], axis=0)

    omega_x = dvz_dy - dvy_dz
    omega_y = dvx_dz - dvz_dx
    omega_z = dvy_dx - dvx_dy

    return np.array([omega_x, omega_y, omega_z]).T

def save_vorticity_with_timestamps(timestamps, vorticities, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)  # Crear el directorio si no existe
    print(f"Saving vorticity data to: {filename}")
    
    try:
        with open(filename, 'w') as f:
            for ts, vort in zip(timestamps, vorticities):
                formatted_output = f"{ts} -> ({vort[0]:.10f}, {vort[1]:.10f}, {vort[2]:.10f})\n"
                f.write(formatted_output)
    except Exception as e:
        print(f"Error writing file {filename}: {e}")

def process_files(input_folder, output_folder, file_suffixes, triads):
    if not os.path.exists(input_folder):
        print(f"Input folder does not exist: {input_folder}")
        return []
    
    generated_files = []  # Lista para almacenar los nombres de los archivos generados
    
    for filename in os.listdir(input_folder):
        if any(filename.endswith(suffix) for suffix in file_suffixes):
            filepath = os.path.join(input_folder, filename)
            print(f"Processing file: {filepath}")
            timestamps, velocities = read_velocity_data(filepath)
            
            if velocities.ndim != 2 or velocities.shape[1] != 6:
                print(f"El archivo {filename} no contiene datos de velocidad válidos. Dimensiones: {velocities.shape}")
                continue
            
            for triad in triads:
                if triad in triad_codes:
                    code = triad_codes[triad]
                    triad_folder = os.path.join(output_folder, code)
                    os.makedirs(triad_folder, exist_ok=True)  # Crear carpeta para la triada si no existe
                    velocities_triad = velocities[:, triad]
                    
                    print(f"Calculating vorticity for triad {code}: {filename}")
                    vorticity = calculate_vorticity(velocities_triad, [0, 1, 2])
                    
                    output_file = f"vorticity_{code}_{filename}"
                    save_vorticity_with_timestamps(timestamps, vorticity, os.path.join(triad_folder, output_file))
                    
                    generated_files.append(f"{os.path.basename(triad_folder)}/{output_file}")

    return generated_files  # Devolver la lista de archivos generados

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab(root_folder, lab_name, file_suffixes, triads):
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
            continue

        input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
        
        # Procesar la línea base
        input_folder = os.path.join(input_folder_base, "Velocity_Baseline")
        output_folder = os.path.join(input_folder_base, "Vorticity_Baseline")
        file_suffixes_baseline = ["negra.txt"]  # Procesar solo archivos "*negra.txt"
        generated_files = process_files(input_folder, output_folder, file_suffixes_baseline, triads)
        print(f"Generated vorticity files for {lab_name}, {shift_name} shift Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        input_folder = os.path.join(input_folder_base, "Velocity_Experimental_Color")
        output_folder = os.path.join(input_folder_base, "Vorticity_Experimental_Color")
        generated_files = process_files(input_folder, output_folder, file_suffixes, triads)
        print(f"Generated vorticity files for {lab_name}, {shift_name} shift Experimental Color:")
        for file in generated_files:
            print(f"  - {file}")

# Define the root folder and suffixes
#file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]

for lab_name in labs:
    process_lab(root_folder, lab_name, file_suffixes, triads)


## 7

$$

    \Huge \text{Helicity Generator by Triads}

$$


In [ ]:
import os
import numpy as np
import re

def read_velocity_data(filepath):
    """
    Lee datos de velocidad desde un archivo.
    """
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    velocity_components = parts[1].strip('()').split(', ')
                    data.append([float(v) for v in velocity_components])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return [], np.array([])

def calculate_helicity(velocities, vorticities, indices, volume):
    """
    Calcula la helicidad en base a las velocidades, vorticidades y volumen.
    """
    selected_velocities = velocities[:, indices]
    helicity = np.einsum('ij,ij->i', selected_velocities, vorticities) * volume
    return helicity

def format_and_save_helicities(timestamps, helicities, filename):
    """
    Formatea y guarda los resultados de helicidad en un archivo.
    """
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    try:
        with open(filename, "w") as file:
            for timestamp, helicity in zip(timestamps, helicities):
                formatted_output = f"{timestamp} -> {helicity:.10f}\n"
                file.write(formatted_output)
    except Exception as e:
        print(f"Error al escribir el archivo {filename}: {e}")

def process_files(input_folder_velocity, input_folder_vorticity, output_folder, file_suffixes, volume, triads):
    """
    Procesa archivos de velocidad y vorticidad para calcular la helicidad y guardarla.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    generated_files = []
    
    for suffix in file_suffixes:
        for filename in os.listdir(input_folder_velocity):
            if filename.endswith(suffix):
                base_filename = filename.split('.')[0]
                filepath_velocity = os.path.join(input_folder_velocity, filename)
                
                for triad in triads:
                    if triad in triad_codes:
                        code = triad_codes[triad]
                        triad_folder = os.path.join(input_folder_vorticity, code)
                        filepath_vorticity = os.path.join(triad_folder, f"vorticity_{code}_{base_filename}.txt")
                        
                        timestamps, velocities = read_velocity_data(filepath_velocity)
                        if velocities.size == 0:
                            continue
                        
                        _, vorticity = read_velocity_data(filepath_vorticity)

                        if vorticity.size == 0:
                            print(f"Skipping file {filename} due to missing vorticity data for triad {code}.")
                            continue
                        
                        helicity = calculate_helicity(velocities, vorticity, triad, volume)
                        
                        output_filename = f"helicity_{code}_{base_filename}.txt"
                        output_path = os.path.join(output_folder, code, output_filename)
                        
                        format_and_save_helicities(timestamps, helicity, output_path)
                        
                        generated_files.append(f"{os.path.basename(output_folder)}/{code}/{output_filename}")

    return generated_files

def find_experiment_folder(root_folder, lab, shift):
    """
    Encuentra la carpeta de experimentos para un laboratorio y turno específico.
    """
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab(root_folder, lab_name, file_suffixes, volume, triads):
    """
    Procesa los archivos de helicidad para un laboratorio específico.
    """
    for shift in ['V', 'M']:
        shift_name = "Vespertino" if shift == 'V' else "Matutino"
        experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
        if not experiment_folder:
            print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
            continue

        input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
        
        if not os.path.exists(input_folder_base):
            print(f"No se puede acceder al directorio: {input_folder_base}")
            continue

        # Procesar la línea base
        input_folder_velocity = os.path.join(input_folder_base, "Velocity_Baseline")
        input_folder_vorticity = os.path.join(input_folder_base, "Vorticity_Baseline")
        output_folder = os.path.join(input_folder_base, "Helicity_Baseline")
        file_suffixes_baseline = ["negra.txt"]  # Procesar solo archivos "*negra.txt"
        generated_files = process_files(input_folder_velocity, input_folder_vorticity, output_folder,file_suffixes_baseline, volume, triads)
        print(f"Archivos de helicidad generados para {lab_name}, turno {shift_name} Baseline:")
        for file in generated_files:
            print(f"  - {file}")

        # Procesar las intervenciones
        input_folder_velocity = os.path.join(input_folder_base, "Velocity_Experimental_Color")
        input_folder_vorticity = os.path.join(input_folder_base, "Vorticity_Experimental_Color")
        output_folder = os.path.join(input_folder_base, "Helicity_Experimental_Color")
        generated_files = process_files(input_folder_velocity, input_folder_vorticity, output_folder, file_suffixes, volume, triads)
        print(f"Archivos de helicidad generados para {lab_name}, turno {shift_name} Experimental Color:")
        for file in generated_files:
            print(f"  - {file}")

# Definir los sufijos de archivos para procesar
#file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]

# Verificar y asignar volúmenes si existen los laboratorios
volumes = {}
if len(labs) > 0:
    volumes[labs[0]] = 7.3195  # Volumen para Lab A (Lab_Betta)
if len(labs) > 1:
    volumes[labs[1]] = 6.8342  # Volumen para Lab B (Lab_Gamma)

# Procesar los laboratorios si existen los volúmenes asignados
for lab_name in labs:
    if lab_name in volumes:  # Verificar que haya volumen asignado para el laboratorio
        process_lab(root_folder, lab_name, file_suffixes, volumes[lab_name], triads)
    else:
        print(f"No se encontró volumen para {lab_name}. Omitiendo laboratorio.")

## 8

$$

    \Huge \text{First Integral of Helicity Generator}

$$


In [ ]:
#Integral Helicidad .txt
import os
import re

def process_file(file_path, output_file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

#@siva se retira la función que normaliza los datos
    
    accumulated_helicity = 0
    results = []
    for line in lines:
        time_stamp, value = line.split('->')
        value = float(value.strip())
        accumulated_helicity += value
        results.append(f"{time_stamp.strip()} -> {accumulated_helicity:.10f}\n")

    with open(output_file_path, 'w') as output_file:
        output_file.writelines(results)

def generate_accumulated_helicity(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for root, dirs, files in os.walk(input_directory):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                output_filename = f"sum_{filename}"
                output_file_path = os.path.join(output_directory, output_filename)
                process_file(file_path, output_file_path)
                print(f"Generated file: {output_filename}")

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab_helicity(root_folder, lab_name, shift, file_suffixes, triad_codes):
    shift_name = "Vespertino" if shift == 'V' else "Matutino"
    experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
    if not experiment_folder:
        print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
        return

    input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
    
    if not os.path.exists(input_folder_base):
        print(f"Skipping {shift_name} shift for {lab_name} as directory does not exist: {input_folder_base}")
        return

    # Procesar los acumulados de Baseline para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Helicity_Baseline", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Helicity_Baseline", triad_code)
        if os.path.exists(input_directory):
            generate_accumulated_helicity(input_directory, output_directory)
            print(f"Generated accumulated helicity files for {lab_name}, {shift_name} shift Baseline, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Baseline Triad {triad_code} as directory does not exist: {input_directory}")

    # Procesar los acumulados de Experimental Color para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Helicity_Experimental_Color", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Helicity_Experimental_Color", triad_code)
        if os.path.exists(input_directory):
            generate_accumulated_helicity(input_directory, output_directory)
            print(f"Generated accumulated helicity files for {lab_name}, {shift_name} shift Experimental Color, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Experimental Color Triad {triad_code} as directory does not exist: {input_directory}")

# Define the root folder, suffixes, and triad codes
#file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]

for lab_name in labs:
    for shift in ['V', 'M']:
        process_lab_helicity(root_folder, lab_name, shift, file_suffixes, triad_codes)

print("Processing complete. All accumulated helicity results are saved in their respective directories.")

## 9

$$

    \Huge \text{First Integral of Absolute Values of Helicity}

$$


In [ ]:
#Integral absoluta helicidad .txt
import os
import re

def process_file_absolute(file_path, output_file_path):
    # Leer las líneas del archivo
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
    #@siva se retira la función que normaliza los datos
    
    # Extraer los valores de helicidad y calcular la suma de los valores absolutos
    absolute_helicity_sum = 0
    results = []
    for line in lines:
        time_stamp, value = line.split('->')
        value = abs(float(value.strip()))  # Uso de valor absoluto
        absolute_helicity_sum += value
        results.append(f"{time_stamp.strip()} -> {absolute_helicity_sum:.10f}\n")

    # Escribir los resultados de la suma de valores absolutos en un nuevo archivo
    with open(output_file_path, 'w') as output_file:
        output_file.writelines(results)

def generate_absolute_helicity_sum(input_directory, output_directory):
    # Crear el directorio de salida si no existe
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Leer todos los archivos en el directorio especificado
    for root, dirs, files in os.walk(input_directory):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                output_filename = f"sum_absolute_{filename}"  # Cambio en el nombre del archivo de salida para reflejar los valores absolutos
                output_file_path = os.path.join(output_directory, output_filename)
                # Procesar el archivo
                process_file_absolute(file_path, output_file_path)
                # Imprimir en consola el nombre del archivo generado
                print(f"Generated file: {output_filename}")

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab_absolute_helicity(root_folder, lab_name, shift, file_suffixes, triad_codes):
    shift_name = "Vespertino" if shift == 'V' else "Matutino"
    experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
    if not experiment_folder:
        print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
        return

    input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
    
    if not os.path.exists(input_folder_base):
        print(f"Skipping {shift_name} shift for {lab_name} as directory does not exist: {input_folder_base}")
        return

    # Procesar los acumulados absolutos de Baseline para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Helicity_Baseline", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Absolute_Helicity_Baseline", triad_code)
        if os.path.exists(input_directory):
            generate_absolute_helicity_sum(input_directory, output_directory)
            print(f"Generated absolute helicity sum files for {lab_name}, {shift_name} shift Baseline, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Baseline Triad {triad_code} as directory does not exist: {input_directory}")

    # Procesar los acumulados absolutos de Experimental Color para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Helicity_Experimental_Color", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Absolute_Helicity_Experimental_Color", triad_code)
        if os.path.exists(input_directory):
            generate_absolute_helicity_sum(input_directory, output_directory)
            print(f"Generated absolute helicity sum files for {lab_name}, {shift_name} shift Experimental Color, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Experimental Color Triad {triad_code} as directory does not exist: {input_directory}")

# Define the root folder, suffixes, and triad codes
#file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]

for lab_name in labs:
    for shift in ['V', 'M']:
        process_lab_absolute_helicity(root_folder, lab_name, shift, file_suffixes, triad_codes)

print("Processing complete. All absolute helicity sum results are saved in their respective directories.")


## 10

$$

    \Huge \text{General Enstrophy File Generator}

$$


In [ ]:
#Enstrofía general .txt
import numpy as np
import os
import re

def read_vorticity_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    vorticity_components = parts[1].strip('()').split(', ')
                    data.append([float(v) for v in vorticity_components])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return [], np.array([])

def calculate_enstrophy(vorticities, volume):
    #@siva se retira la función que normaliza los datos
    # Enstrophy calculation: sum of the squares of vorticity components, multiplied by volume
    enstrophy = np.sum(vorticities**2, axis=1) * volume
    return enstrophy

def format_and_save_enstrophies(timestamps, enstrophies, filename):
    try:
        with open(filename, "w") as file:
            for timestamp, enstrophy in zip(timestamps, enstrophies):
                formatted_output = f"{timestamp} -> {enstrophy:.10f}\n"
                file.write(formatted_output)
    except Exception as e:
        print(f"Error writing file {filename}: {e}")

def process_files_enstrophy(input_folder, output_folder, file_suffixes, volume):
    os.makedirs(output_folder, exist_ok=True)
    generated_files = []  # Lista para almacenar los nombres de los archivos generados

    for root, dirs, files in os.walk(input_folder):
        for filename in files:
            for file_suffix in file_suffixes:
                if filename.endswith(file_suffix):
                    base_filename = filename[:-len(file_suffix)]  # Correctly removing the suffix
                    
                    filepath_vorticity = os.path.join(root, filename)
                    
                    timestamps, vorticity = read_vorticity_data(filepath_vorticity)
                    if vorticity.size == 0:
                        continue
                    
                    enstrophy = calculate_enstrophy(vorticity, volume)
                    
                    output_filename = f"enstrophy_{base_filename}{file_suffix}"
                    output_filepath = os.path.join(output_folder, output_filename)
                    
                    format_and_save_enstrophies(timestamps, enstrophy, output_filepath)
                    generated_files.append(f"{os.path.basename(output_folder)}/{output_filename}")

    return generated_files  # Devolver la lista de archivos generados

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab_enstrophy(root_folder, lab_name, shift, file_suffixes, volume, triad_codes):
    shift_name = "Vespertino" if shift == 'V' else "Matutino"
    experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
    if not experiment_folder:
        print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
        return

    input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
    
    if not os.path.exists(input_folder_base):
        print(f"Skipping {shift_name} shift for {lab_name} as directory does not exist: {input_folder_base}")
        return

    # Procesar enstrofia de Baseline para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Vorticity_Baseline", triad_code)
        output_directory = os.path.join(input_folder_base, "Enstrophy_Baseline", triad_code)
        if os.path.exists(input_directory):
            generated_files = process_files_enstrophy(input_directory, output_directory, file_suffixes, volume)
            print(f"Generated enstrophy files for {lab_name}, {shift_name} shift Baseline, Triad {triad_code}:")
            for file in generated_files:
                print(f"  - {file}")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Baseline Triad {triad_code} as directory does not exist: {input_directory}")

    # Procesar enstrofia de Experimental Color para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Vorticity_Experimental_Color", triad_code)
        output_directory = os.path.join(input_folder_base, "Enstrophy_Experimental_Color", triad_code)
        if os.path.exists(input_directory):
            generated_files = process_files_enstrophy(input_directory, output_directory, file_suffixes, volume)
            print(f"Generated enstrophy files for {lab_name}, {shift_name} shift Experimental Color, Triad {triad_code}:")
            for file in generated_files:
                print(f"  - {file}")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Experimental Color Triad {triad_code} as directory does not exist: {input_directory}")

# Define the root folder, suffixes, and triad codes
#file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]

# Verificar y asignar volúmenes si existen los laboratorios
volumes = {}
if len(labs) > 0:
    volumes[labs[0]] = 7.3195  # Volumen para Lab A (Lab_Betta)
if len(labs) > 1:
    volumes[labs[1]] = 6.8342  # Volumen para Lab B (Lab_Gamma)
    
for lab_name in labs:
    for shift in ['V', 'M']:
        process_lab_enstrophy(root_folder, lab_name, shift, file_suffixes, volumes[lab_name], triad_codes)

print("Processing complete. All enstrophy results are saved in their respective directories.")

## 11

$$

    \Huge \text{First Integral of Enstrophy}

$$


In [ ]:
#Integral Enstrofía
import os
import numpy as np
import re

def read_vorticity_data(filepath):
    data = []
    timestamps = []
    try:
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(' -> ')
                if len(parts) > 1:
                    timestamps.append(parts[0])
                    vorticity_components = parts[1].strip('()').split(', ')
                    data.append([float(v) for v in vorticity_components])
        return timestamps, np.array(data)
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return [], np.array([])

def calculate_enstrophy(vorticities, volume):
    # Enstrophy calculation: sum of the squares of vorticity components, multiplied by volume
    enstrophy = np.sum(vorticities**2, axis=1) * volume
    return enstrophy

def format_and_save_enstrophies(timestamps, enstrophies, filename):
    try:
        with open(filename, "w") as file:
            for timestamp, enstrophy in zip(timestamps, enstrophies):
                formatted_output = f"{timestamp} -> {enstrophy:.10f}\n"
                file.write(formatted_output)
    except Exception as e:
        print(f"Error writing file {filename}: {e}")

def process_files_enstrophy(input_folder, output_folder, file_suffixes, volume):
    os.makedirs(output_folder, exist_ok=True)
    generated_files = []  # Lista para almacenar los nombres de los archivos generados

    for filename in os.listdir(input_folder):
        for file_suffix in file_suffixes:
            if filename.endswith(file_suffix):
                base_filename = filename[:-len(file_suffix)]  # Correctly removing the suffix
                
                filepath_vorticity = os.path.join(input_folder, filename)
                
                timestamps, vorticity = read_vorticity_data(filepath_vorticity)
                if vorticity.size == 0:
                    continue
                
                enstrophy = calculate_enstrophy(vorticity, volume)
                
                output_filename = f"enstrophy_{base_filename}{file_suffix}"
                output_filepath = os.path.join(output_folder, output_filename)
                
                format_and_save_enstrophies(timestamps, enstrophy, output_filepath)
                generated_files.append(f"{os.path.basename(output_folder)}/{output_filename}")

    return generated_files  # Devolver la lista de archivos generados

def accumulate_enstrophy(file_path, output_file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        accumulated_enstrophy = 0
        results = []
        for line in lines:
            time_stamp, value = line.split('->')
            value = float(value.strip())
            accumulated_enstrophy += value
            results.append(f"{time_stamp.strip()} -> {accumulated_enstrophy:.10f}\n")

        with open(output_file_path, 'w') as output_file:
            output_file.writelines(results)
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

def generate_accumulated_enstrophy(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for root, dirs, files in os.walk(input_directory):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                output_filename = f"sum_{filename}"
                output_file_path = os.path.join(output_directory, output_filename)
                accumulate_enstrophy(file_path, output_file_path)
                print(f"Generated file: {output_filename}")

def find_experiment_folder(root_folder, lab, shift):
    date_regex = re.compile(r'\d{2}[A-Za-z]{3}\d{2}')
    for folder in os.listdir(root_folder):
        if date_regex.search(folder) and lab in folder:
            for subfolder in os.listdir(os.path.join(root_folder, folder)):
                if date_regex.search(subfolder) and shift in subfolder:
                    return os.path.join(root_folder, folder, subfolder), date_regex.search(folder).group(0)
    return None, None

def process_lab_accumulated_enstrophy(root_folder, lab_name, shift, file_suffixes, volume, triad_codes):
    shift_name = "Vespertino" if shift == 'V' else "Matutino"
    experiment_folder, date_str = find_experiment_folder(root_folder, lab_name, shift)
    if not experiment_folder:
        print(f"No se encontró ningún directorio para {lab_name}, {shift}.")
        return

    input_folder_base = os.path.join(experiment_folder, "Data Analysis", "Processing Data")
    
    if not os.path.exists(input_folder_base):
        print(f"Skipping {shift_name} shift for {lab_name} as directory does not exist: {input_folder_base}")
        return

    # Procesar enstrofia acumulada de Baseline para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Enstrophy_Baseline", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Enstrophy_Baseline", triad_code)
        if os.path.exists(input_directory):
            generate_accumulated_enstrophy(input_directory, output_directory)
            print(f"Generated accumulated enstrophy files for {lab_name}, {shift_name} shift Baseline, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Baseline Triad {triad_code} as directory does not exist: {input_directory}")

    # Procesar enstrofia acumulada de Experimental Color para cada triada
    for triad, triad_code in triad_codes.items():
        input_directory = os.path.join(input_folder_base, "Enstrophy_Experimental_Color", triad_code)
        output_directory = os.path.join(input_folder_base, "Sum_Enstrophy_Experimental_Color", triad_code)
        if os.path.exists(input_directory):
            generate_accumulated_enstrophy(input_directory, output_directory)
            print(f"Generated accumulated enstrophy files for {lab_name}, {shift_name} shift Experimental Color, Triad {triad_code}.")
        else:
            print(f"Skipping {shift_name} shift for {lab_name} Experimental Color Triad {triad_code} as directory does not exist: {input_directory}")

# Define the root folder, suffixes, and triad codes
#file_suffixes = ["medroja.txt", "medmorada.txt", "medazul.txt", "medverde.txt", "medamarilla.txt", "mednegra.txt"]

# Verificar y asignar volúmenes si existen los laboratorios
volumes = {}
if len(labs) > 0:
    volumes[labs[0]] = 7.3195  # Volumen para Lab A (Lab_Betta)
if len(labs) > 1:
    volumes[labs[1]] = 6.8342  # Volumen para Lab B (Lab_Gamma)

for lab_name in labs:
    for shift in ['V', 'M']:
        process_lab_accumulated_enstrophy(root_folder, lab_name, shift, file_suffixes, volumes[lab_name], triad_codes)

print("Processing complete. All accumulated enstrophy results are saved in their respective directories.")

Códigos pesajes
----------------

In [ ]:
import os
import re
import re

def extraer_fecha(ruta):
    # Definir el patrón para coincidir con el formato de fecha ddMmmYY (como 12Ago24)
    patron_fecha = r"\d{2}[A-Za-z]{3}\d{2}"
    
    # Buscar la fecha en la cadena
    coincidencia = re.search(patron_fecha, ruta)
    
    if coincidencia:
        return coincidencia.group()  # Devolver la fecha encontrada
    else:
        return None  # Si no se encuentra, devolver None
    
# Función para decodificar el texto hexadecimal
def decodificar_hex(hex_string):
    try:
        hex_cleaned = hex_string.strip().replace(' ', '')
        decoded_string = bytes.fromhex(hex_cleaned).decode('ascii', errors='ignore').strip()
        return decoded_string
    except Exception as e:
        print(f"Error decodificando: {e}")
        return None

# Función para procesar el archivo y extraer la hora y el valor limpio
def procesar_archivo_para_hora_y_texto(ruta_archivo):
    resultados = []
    
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            match_hora = re.search(r'(\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}\.\d{3})', linea)
            if match_hora:
                hora = match_hora.group(1).split(' ')[1]  # Obtener solo la parte de la hora
                partes = linea.strip().split(',')
                if len(partes) > 3:
                    hex_data = partes[-1]
                    decoded_value = decodificar_hex(hex_data)

                    if decoded_value:
                        resultados.append(f"{hora}, {decoded_value}")
                    else:
                        resultados.append(f"{hora}, .")
    
    return resultados

# Función para guardar los resultados en un archivo de salida
def guardar_resultados(ruta_salida, resultados):
    with open(ruta_salida, 'w') as archivo_salida:
        for resultado in resultados:
            archivo_salida.write(resultado + '\n')

# Función para construir la ruta del archivo a procesar
def construir_ruta(root_folder, fecha, lab, turno, color, numero):
    colorin = color.lower()
    return os.path.join(root_folder, f"{fecha} - {lab}", f"{fecha}.{turno} - {lab}", "Pesajes", f"{numero:02d}med{colorin}.log")

# Función para filtrar resultados
def filtrar_resultados(resultados):
    # Expresión regular para el formato deseado
    pattern = r'^\d{2}:\d{2}:\d{2}\.\d{3}, \d+\.\d{2}g$'
    return [resultado for resultado in resultados if re.match(pattern, resultado.strip())]

# Si este es el archivo principal
if __name__ == "__main__":
    #print(root_folder)
    fecha = extraer_fecha(root_folder)  # Asegúrate de definir esta función
    turnos = ["M", "V"]
    colores = ["Amarilla", "Roja", "Azul", "Verde", "Morada", "Negra", "Naranja"]
    numeros_archivo = range(1, 11)

    for lab in labs:
        for turno in turnos:
            for color in colores:
                for numero in numeros_archivo:
                    ruta_archivo = construir_ruta(root_folder, fecha, lab, turno, color, numero)
                    print(ruta_archivo)
                    
                    colorin = color.lower()
                    if os.path.exists(ruta_archivo):
                        ruta_salida = os.path.join(os.path.dirname(ruta_archivo), f"{numero:02d}med{colorin}.txt")

                        print(f"\nProcesando para Lab: {lab}, Turno: {turno}, Color: {color}, Archivo: {numero:02d}")
                        print(f"Archivo a procesar: {ruta_archivo}")
                        
                        # Procesar el archivo para extraer hora y texto limpio
                        resultados = procesar_archivo_para_hora_y_texto(ruta_archivo)

                        # Filtrar resultados
                        resultados_filtrados = filtrar_resultados(resultados)

                        # Guardar los resultados filtrados en el archivo de salida
                        guardar_resultados(ruta_salida, resultados_filtrados)
                        print(f"Archivo encontrado: {ruta_archivo}")
                    #else:
                     #   print(f"Archivo no encontrado: {ruta_archivo}")
    print("\nProcesamiento completo.")

In [ ]:
import os
import re
import pandas as pd

# Función para extraer la fecha del nombre del directorio raíz
def extraer_fecha(root_folder):
    nombre_directorio = os.path.basename(root_folder)
    match = re.search(r'(\d{2}[A-Za-z]{3}\d{2})', nombre_directorio)
    
    if match:
        fecha_str = match.group(1)  # Captura la fecha en formato '16Sep24'
        return fecha_str
    else:
        raise ValueError(f"No se encontró una fecha válida en el nombre del directorio {root_folder}.")

# Función para cargar y procesar los datos
def cargar_y_procesar_datos(ruta_archivo):
    datos_procesados = []
    
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            partes = linea.split(',')
            hora = partes[0].strip()  # Extrae la hora
            valor = partes[1].strip()  # Extrae el valor con 'g'
            
            # Elimina la 'g' y convierte a float
            valor_numero = float(valor.replace('g', ''))
            # Aplica el tratamiento Dato * 9.81 / 5.00
            valor_procesado = valor_numero * 9.81 / 5.00
            
            # Guarda la hora y el valor procesado
            datos_procesados.append(f"{hora}, {valor_procesado:.2f}\n")
    
    return datos_procesados

# Función para guardar los datos procesados en un archivo .txt
def guardar_datos_txt(datos_procesados, ruta_salida):
    with open(ruta_salida, 'w') as archivo_salida:
        archivo_salida.writelines(datos_procesados)
    print(f"Datos procesados guardados en: {ruta_salida}")

# Función principal para procesar y guardar archivos
def procesar_archivos(root_folder):
    turnos = ["M", "V"]
    colores = ["Amarilla", "Roja", "Azul", "Verde", "Morada", "Negra", "Naranja"]
    numeros_archivo = range(1, 11)

    fecha = extraer_fecha(root_folder)

    for lab in labs:
        for turno in turnos:
            for color in colores:
                for numero in numeros_archivo:
                    colorin = color.lower()
                    ruta_archivo = os.path.join(root_folder, f"{fecha} - Lab_{lab}", f"{fecha}.{turno} - Lab_{lab}", "Pesajes", f"0{numero}med{colorin}.txt")
                    
                    if os.path.exists(ruta_archivo):
                        # Cargar y procesar los datos del archivo
                        datos_procesados = cargar_y_procesar_datos(ruta_archivo)

                        # Definir la carpeta y el nombre del archivo de salida
                        output_folder = os.path.join(root_folder, f"{fecha} - Lab_{lab}", f"{fecha}.{turno} - Lab_{lab}", "Pesajes", "procesados")
                        os.makedirs(output_folder, exist_ok=True)
                        ruta_salida = os.path.join(output_folder, f"resultados_0{numero}_{colorin}.txt")

                        print(ruta_salida)

                        # Guardar los datos procesados en un archivo .txt
                        guardar_datos_txt(datos_procesados, ruta_salida)
                    #else:
                        print(f"Archivo no encontrado: {ruta_archivo}")

# Si este es el archivo principal
if __name__ == "__main__":
    procesar_archivos(root_folder)
    print("\nProcesamiento completo.")

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Definir un diccionario para mapear los nombres de los colores a los colores de matplotlib
color_map = {
    "amarilla": "yellow",
    "roja": "red",
    "azul": "blue",
    "verde": "green",
    "morada": "purple",
    "negra": "black",
    "naranja": "orange"
}

# Función para cargar datos y crear DataFrame
def cargar_datos(ruta_archivo):
    # Leer archivo en un DataFrame
    df = pd.read_csv(ruta_archivo, header=None, names=["Timestamp", "Valor"])
    
    # Convertir el timestamp a formato de tiempo de pandas para facilitar el graficado
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S.%f')
    
    return df

# Función para graficar serie de tiempo como scatter plot con color especificado
def graficar_serie_tiempo(df, titulo, ruta_salida, color):
    plt.figure(figsize=(10, 6))
    plt.scatter(df['Timestamp'], df['Valor'], color=color)
    plt.xlabel('Tiempo')
    plt.ylabel('Valor')
    plt.title(titulo)
    plt.grid()
    plt.xticks(rotation=45)
    
    # Guardar la gráfica en la ruta especificada
    plt.savefig(ruta_salida)
    plt.close()  # Cierra la gráfica para no mostrarla en la terminal
    print(f"Gráfica guardada en: {ruta_salida}")

# Función principal para procesar y graficar
def procesar_y_graficar_archivos(root_folder):
    fecha = extraer_fecha(root_folder)  # Usar la función extraer_fecha definida previamente
    turnos = ["M", "V"]
    #colores = ["Amarilla", "Roja", "Azul", "Verde", "Morada", "Negra"]
    numeros_archivo = range(1, 11)

    for lab in labs:
        for turno in turnos:
            for color in colores:
                for numero in numeros_archivo:
                    colorin = color.lower()
                    ruta_archivo = os.path.join(root_folder, f"{fecha} - {lab}", f"{fecha}.{turno} - {lab}", "Pesajes", f"0{numero}med{colorin}.txt")
                    
                    if os.path.exists(ruta_archivo):
                        # Cargar los datos del archivo procesado
                        df = cargar_datos(ruta_archivo)
                        
                        # Crear el título de la gráfica
                        titulo = f"Serie de Tiempo - Lab {lab} - Turno {turno} - {color} - Medición {numero}"
                        
                        # Definir la carpeta y el nombre del archivo de salida para la gráfica
                        output_folder = os.path.join(root_folder, f"{fecha} - {lab}", f"{fecha}.{turno} - {lab}", "Pesajes", "graficas")
                        os.makedirs(output_folder, exist_ok=True)
                        ruta_salida_grafica = os.path.join(output_folder, f"grafica_0{numero}_{colorin}.png")
                        
                        # Obtener el color correspondiente a partir del nombre del archivo
                        color_grafica = color_map.get(colorin, 'blue')  # Default a 'blue' si no se encuentra el color
                        
                        # Graficar la serie de tiempo como scatter plot con el color correspondiente
                        graficar_serie_tiempo(df, titulo, ruta_salida_grafica, color_grafica)
                    else:
                        print(f"Archivo no encontrado: {ruta_archivo}")

# Ejecutar el procesamiento y graficado
if __name__ == "__main__":
    procesar_y_graficar_archivos(root_folder)
    print("\nProcesamiento y graficado completo.")

Códigos de Comparación de Modulos
----------------------

In [ ]:
#@siva las siguientes celdas de código generan la comparación de modulos
fechas = []

# Función para extraer la fecha del nombre del directorio raíz
def extraer_fecha(root_folder):
    nombre_directorio = os.path.basename(root_folder)
    match = re.search(r'(\d{2}[A-Za-z]{3}\d{2})', nombre_directorio)
    
    if match:
        fecha_str = match.group(1)  # Captura la fecha en formato '16Sep24'
        return fecha_str
    else:
        raise ValueError(f"No se encontró una fecha válida en el nombre del directorio {root_folder}.")

fechas.append(extraer_fecha(root_folder))

print(fechas)

In [ ]:
import os
import re  # Para utilizar expresiones regulares

# Listas para determinar laboratorio, turno y color
turnos = ['M', 'V']
colores = ['Amarilla', 'Roja', 'Verde', 'Morada', 'Azul', 'Negra', 'Naranja']

# Diccionario para almacenar archivos según el criterio
archivos_validos = {
    lab: {turno: {'experimental': [], 'base': []} for turno in turnos} for lab in labs
}

# Función para extraer el número del archivo desde su nombre
def extraer_numero_archivo(nombre_archivo):
    match = re.match(r'(\d+)', nombre_archivo)  # Busca el número al inicio del nombre
    return int(match.group(1)) if match else None

# Función para buscar y analizar archivos según los criterios específicos para cada fecha
def buscar_archivos_y_analizar(root_folder, fechas):
    archivos_info = []  # Lista para almacenar la información de todos los archivos procesados

    for fecha in fechas:  # Iterar sobre cada fecha en la lista
        for lab in labs:
            for turno in turnos:
                for color in colores:
                    # Definir la ruta base específica para cada fecha con el subdirectorio de la fecha
                    ruta_base = os.path.join(root_folder, f'{fecha} - {lab}', f'{fecha}.{turno} - {lab}', color)
                    # Verificar la existencia de la ruta
                    if not os.path.exists(ruta_base):
                        continue
                    
                    # Recorrer los archivos en la ruta base
                    for root, _, files in os.walk(ruta_base):
                        if not files:  # Verificar si no hay archivos en la carpeta
                            print(f"No se encontraron archivos en: {ruta_base}")
                            continue

                        for file in files:
                            if file.endswith('.txt') and '00' not in file:
                                filepath = os.path.join(root, file)

                                # Extraer el número del archivo
                                numero_archivo = extraer_numero_archivo(file)

                                tipo = 'base' if color.lower() == 'negra' else 'experimental'
                                archivos_validos[lab][turno][tipo].append({
                                    'Número': numero_archivo,  # Almacena el número extraído del nombre del archivo
                                    'Ruta': filepath,
                                    'Fecha': fecha  # Añadir la fecha al diccionario
                                })

                                archivos_info.append({
                                    'Número': numero_archivo,
                                    'Archivo': file,
                                    'Laboratorio': lab,
                                    'Turno': turno,
                                    'Tipo': 'Base' if color.lower() == 'negra' else 'Experimental',
                                    'Fecha': fecha  # Añadir la fecha al diccionario
                                })
    return archivos_info

# Llamada a la función de análisis para todas las fechas
archivos_info = buscar_archivos_y_analizar(root_folder, fechas)

# Mostrar la información detallada de cada archivo procesado (descomentar para visualización)
# for info in archivos_info:
#     print(info)

# También puedes acceder a los archivos válidos por laboratorio, turno y tipo de archivo así:
print(archivos_validos)
# print(archivos_validos['Lab_Gamma']['M'])


In [ ]:
#@siva en base a los archivos que sean adecuados, genera comparaciones de los datos crudos

import os
import numpy as np
import pandas as pd

# Función para calcular módulos y sumar utilizando el diccionario `archivos_validos` con fechas
def calcular_modulos_y_sumar_con_fechas(archivos_validos):
    resultados = {}
    total_casos_satisfactorios = 0  # Inicializar el contador total de casos satisfactorios

    for lab in archivos_validos.keys():  # Iterar sobre cada laboratorio
        for turno in archivos_validos[lab].keys():  # Iterar sobre cada turno
            experimental_archivos = archivos_validos[lab][turno]['experimental']
            base_archivos = archivos_validos[lab][turno]['base']
            
            # Crear un diccionario para acceder a los archivos base por número
            base_dict = {archivo['Número']: archivo['Ruta'] for archivo in base_archivos}

            for exp_archivo in experimental_archivos:
                exp_num = exp_archivo['Número']
                exp_ruta = exp_archivo['Ruta']
                exp_nombre = os.path.basename(exp_ruta)  # Obtener el nombre del archivo sin la ruta
                exp_fecha = exp_archivo['Fecha']  # Obtener la fecha del archivo experimental

                # Verificar si `exp_num` es válido
                if exp_num is None:
                    print(f"Advertencia: 'Número' es None para el archivo {exp_ruta}. Saltando este archivo.")
                    continue

                # Calcular la suma de módulos para el archivo experimental
                mod_exp_suma = sumar_modulos(exp_ruta)

                # Inicializar la lista de resultados para cada archivo experimental
                if exp_fecha not in resultados:
                    resultados[exp_fecha] = {}  # Crear entrada para la fecha si no existe

                # Crear una lista para almacenar los resultados por laboratorio, turno y archivo
                resultados[exp_fecha][f'{lab}_{turno}_Archivo_{exp_num}'] = []

                # Verificar si hay un archivo base con el mismo número y el siguiente
                for num in (exp_num, exp_num + 1):
                    if num in base_dict:
                        base_ruta = base_dict[num]  # Accedemos a la ruta correctamente
                        #print(exp_ruta)
                        mod_base_suma = sumar_modulos(base_ruta)
                        dif_porcentual = (abs(mod_exp_suma - mod_base_suma) / mod_base_suma) * 100

                        # Determinar el resultado basado en la diferencia porcentual
                        if dif_porcentual < 10:
                            resultado = "No hay evidencia de intervención."
                        elif 10 <= dif_porcentual <= 33.9:
                            resultado = "Resultados inconclusos."
                        else:  # valor > 33.9
                            resultado = "Evidencia de intervención satisfactoria."
                            total_casos_satisfactorios += 1  # Aumentar contador total de casos satisfactorios

                        # Obtener el nombre del archivo base sin la ruta
                        base_nombre = os.path.basename(base_ruta)

                        # Almacenar el resultado en la lista de resultados para el archivo experimental
                        resultados[exp_fecha][f'{lab}_{turno}_Archivo_{exp_num}'].append({
                            'Laboratorio': lab,  # Agregar el nombre del laboratorio
                            'Archivo_Experimental': exp_nombre,
                            'Archivo_Base': base_nombre,
                            'Modulo_Experimental': mod_exp_suma,
                            'Modulo_Base': mod_base_suma,
                            'Diferencia_Porcentual': dif_porcentual,
                            'Resultado': resultado
                        })

    return resultados, total_casos_satisfactorios

# Función para calcular la suma de módulos de los vectores en un archivo
def sumar_modulos(filepath):
    suma_modulos = 0

    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                vector = parts[1].strip('()').split(',')
                valores = np.array([float(num.strip()) for num in vector])
                modulo = np.linalg.norm(valores)  # Calcular el módulo del vector
                suma_modulos += modulo

    return suma_modulos

# Función para guardar los resultados en un archivo Excel separado por cada fecha dentro de 'Resultados_Excel' en root_folder
def guardar_resultados_por_fecha(resultados, root_folder):
    # Crear la carpeta 'Resultados_Excel' dentro de `root_folder` si no existe
    carpeta_resultados = os.path.join(root_folder, "Excels", 'Moduli Analysis')
    os.makedirs(carpeta_resultados, exist_ok=True)

    for fecha, datos_fecha in resultados.items():
        # Crear una lista para almacenar las filas del DataFrame
        filas = []

        for archivo, lista_resultados in datos_fecha.items():
            for resultado in lista_resultados:
                filas.append({
                    'Laboratorio': resultado['Laboratorio'],
                    'Archivo_Experimental': resultado['Archivo_Experimental'],
                    'Archivo_Base': resultado['Archivo_Base'],
                    'Modulo_Experimental': resultado['Modulo_Experimental'],
                    'Modulo_Base': resultado['Modulo_Base'],
                    'Diferencia_Porcentual': resultado['Diferencia_Porcentual'],
                    'Resultado': resultado['Resultado']
                })

        # Crear un DataFrame a partir de las filas
        df = pd.DataFrame(filas)

        # Definir el nombre del archivo Excel basado en la fecha y guardarlo en 'Resultados_Excel'
        nombre_archivo = f'Moduli_intervencion_general_{fecha}.xlsx'
        ruta_archivo = os.path.join(carpeta_resultados, nombre_archivo)

        # Guardar el DataFrame en un archivo Excel
        df.to_excel(ruta_archivo, index=False)
        print(f"Resultados guardados en {ruta_archivo}")

    return carpeta_resultados

# Función para guardar el total de casos satisfactorios en un archivo .txt
def guardar_casos_satisfactorios_txt(total_casos_satisfactorios, carpeta_resultados):
    # Definir la ruta del archivo .txt para guardar el total de casos satisfactorios en la carpeta de resultados Excel
    ruta_txt = os.path.join(carpeta_resultados, 'total_casos_satisfactorios.txt')

    with open(ruta_txt, 'w') as file:
        file.write(f'Total de casos satisfactorios: {total_casos_satisfactorios}\n')
        print(f"Total de casos satisfactorios guardado en {ruta_txt}")

# Llamada a la función para calcular módulos y sumar utilizando `archivos_validos`
resultados, total_casos_satisfactorios = calcular_modulos_y_sumar_con_fechas(archivos_validos)

# Guardar los resultados en archivos Excel separados por fecha dentro de la carpeta `root_folder/Resultados_Excel`
carpeta_resultados = guardar_resultados_por_fecha(resultados, root_folder)

# Guardar el total de casos satisfactorios en un archivo .txt en la misma carpeta donde están los Excel
guardar_casos_satisfactorios_txt(total_casos_satisfactorios, carpeta_resultados)

print("Resultados guardados por fecha en archivos Excel separados dentro de 'Resultados_Excel'.")
print("Total de casos satisfactorios guardado en 'total_casos_satisfactorios.txt' en la misma carpeta de resultados.")

In [ ]:
#@siva crea comparacion de la primer integral helicidad
import os
import numpy as np
import glob
import pandas as pd

# Definir la lista de triadas
triads = [
    "FRT", "PLB", "FLT", "PRB", "FRB",
    "PLT", "FLB", "PRT", "RTB", "FLP",
    "LTB", "FRP"
]

# Función para calcular módulos y sumar utilizando el diccionario `archivos_validos` con fechas
def calcular_modulos_y_sumar_con_fechas(archivos_validos, root_folder):
    resultados_por_fecha = {}  # Diccionario para almacenar los resultados por fecha
    total_casos_satisfactorios = 0  # Contador total de casos satisfactorios

    for lab in archivos_validos.keys():
        for turno in archivos_validos[lab].keys():
            for triad in triads:
                experimental_archivos = archivos_validos[lab][turno]['experimental']
                base_archivos = archivos_validos[lab][turno]['base']
                
                # Crear un diccionario para acceder a los archivos base por número
                base_dict = {archivo['Número']: archivo['Ruta'] for archivo in base_archivos}

                for exp_archivo in experimental_archivos:
                    exp_num = exp_archivo['Número']
                    exp_fecha = exp_archivo['Fecha']  # Obtener la fecha del archivo experimental
                    exp_ruta_pattern = os.path.join(root_folder, f"{exp_fecha} - {lab}", f"{exp_fecha}.{turno} - {lab}", "Data Analysis", "Processing Data", "Sum_Helicity_Experimental_Color", triad, f"sum_helicity_{triad}_velocity_0{exp_num}med*.txt")
                    
                    # Usar glob para buscar el archivo experimental
                    exp_archivos_encontrados = glob.glob(exp_ruta_pattern)

                    if not exp_archivos_encontrados:
                        continue
                    
                    exp_ruta = exp_archivos_encontrados[0]  # Toma el primer archivo encontrado
                    exp_nombre = os.path.basename(exp_ruta)  # Obtener el nombre del archivo sin la ruta

                    # Calcular la suma de módulos para el archivo experimental
                    mod_exp_suma = sumar_modulos(exp_ruta)

                    # Inicializar el diccionario para almacenar los resultados por fecha si no existe
                    if exp_fecha not in resultados_por_fecha:
                        resultados_por_fecha[exp_fecha] = {triad: [] for triad in triads}

                    # Verificar si hay un archivo base con el mismo número y el siguiente
                    for num in (exp_num, exp_num + 1):
                        if num in base_dict:
                            base_ruta_pattern = os.path.join(root_folder, f"{exp_fecha} - {lab}", f"{exp_fecha}.{turno} - {lab}", "Data Analysis", "Processing Data", "Sum_Helicity_Baseline", triad, f"sum_helicity_{triad}_velocity_0{num}med*.txt")
                            
                            # Usar glob para buscar el archivo base
                            base_archivos_encontrados = glob.glob(base_ruta_pattern)
                            #print(base_archivos_encontrados)
                            if not base_archivos_encontrados:
                                print(f"No se encontró archivo base para el patrón: {base_ruta_pattern}")
                                continue

                            base_ruta = base_archivos_encontrados[0]  # Toma el primer archivo base encontrado
                            base_nombre = os.path.basename(base_ruta)  # Obtener el nombre del archivo base sin la ruta
                            
                            # Calcular la suma de módulos para el archivo base
                            mod_base_suma = sumar_modulos(base_ruta)
                            dif_porcentual = (abs(mod_exp_suma - mod_base_suma) / mod_base_suma) * 100

                            # Determinar el resultado basado en la diferencia porcentual
                            if dif_porcentual < 10:
                                resultado = "No hay evidencia de intervención."
                            elif 10 <= dif_porcentual <= 33.9:
                                resultado = "Resultados inconclusos."
                            else:  # valor > 33.9
                                resultado = "Evidencia de intervención satisfactoria."
                                total_casos_satisfactorios += 1  # Aumentar contador total de casos satisfactorios

                            # Almacenar el resultado en la lista de resultados para la triada y fecha
                            #print(resultados_por_fecha[exp_fecha][triad])
                            resultados_por_fecha[exp_fecha][triad].append({
                                'Laboratorio': lab,
                                'Turno': turno,
                                'Archivo_Experimental': exp_nombre,
                                'Archivo_Base': base_nombre,
                                'Modulo_Experimental': mod_exp_suma,
                                'Modulo_Base': mod_base_suma,
                                'Diferencia_Porcentual': dif_porcentual,
                                'Resultado': resultado
                            })

    return resultados_por_fecha, total_casos_satisfactorios

# Función para calcular la suma de módulos de los vectores en un archivo
def sumar_modulos(filepath):
    suma_modulos = 0

    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                vector = parts[1].strip('()').split(',')
                valores = np.array([float(num.strip()) for num in vector])
                modulo = np.linalg.norm(valores)  # Calcular el módulo del vector
                suma_modulos += modulo

    return suma_modulos

# Función para guardar los resultados en un archivo Excel separado por cada fecha dentro de `Resultados_Triadas` en `root_folder`
def guardar_resultados_por_fecha(resultados_por_fecha, root_folder):
    # Crear la carpeta 'Resultados_Triadas_Helicidad' dentro de `root_folder` si no existe
    carpeta_resultados = os.path.join(root_folder, "Excels", 'Helicity Analysis')
    os.makedirs(carpeta_resultados, exist_ok=True)

    for fecha, resultados_triadas in resultados_por_fecha.items():
        # Crear un archivo Excel con hojas por triada para cada fecha
        nombre_archivo = f'resultados_triadas_{fecha}.xlsx'
        ruta_archivo = os.path.join(carpeta_resultados, nombre_archivo)

        with pd.ExcelWriter(ruta_archivo, engine='openpyxl') as writer:
            for triad, resultados_triadas_data in resultados_triadas.items():
                # Crear un DataFrame para cada triada y guardarlo en una hoja
                df = pd.DataFrame(resultados_triadas_data)
                if not df.empty:
                    # Ordenar columnas según el código anterior
                    df = df[['Laboratorio', 'Turno', 'Archivo_Experimental', 'Archivo_Base', 'Modulo_Experimental', 'Modulo_Base', 'Diferencia_Porcentual', 'Resultado']]
                df.to_excel(writer, sheet_name=triad, index=False)

        print(f"Resultados guardados en {ruta_archivo}")

    return carpeta_resultados

# Función para guardar el total de casos satisfactorios en un archivo .txt
def guardar_casos_satisfactorios_txt(total_casos_satisfactorios, carpeta_resultados):
    # Definir la ruta del archivo .txt para guardar el total de casos satisfactorios en la carpeta de resultados Excel
    ruta_txt = os.path.join(carpeta_resultados, 'total_casos_satisfactorios.txt')

    with open(ruta_txt, 'w') as file:
        file.write(f'Total de casos satisfactorios: {total_casos_satisfactorios}\n')
        print(f"Total de casos satisfactorios guardado en {ruta_txt}")

# Definir la ruta `root_folder` según tu entorno
# root_folder = 'E:/Filter Data Agos-Sep'  # Reemplaza esta línea con tu `root_folder`

# Llamada a la función para calcular módulos y sumar utilizando `archivos_validos`
resultados_por_fecha, total_casos_satisfactorios = calcular_modulos_y_sumar_con_fechas(archivos_validos, root_folder)

# Guardar los resultados en archivos Excel separados por fecha dentro de la carpeta `root_folder/Resultados_Triadas_Helicidad`
carpeta_resultados = guardar_resultados_por_fecha(resultados_por_fecha, root_folder)

# Guardar el total de casos satisfactorios en un archivo .txt en la misma carpeta donde están los Excel
guardar_casos_satisfactorios_txt(total_casos_satisfactorios, carpeta_resultados)

print("Resultados guardados por fecha en archivos Excel separados dentro de 'Resultados_Triadas_Helicidad'.")
print("Total de casos satisfactorios guardado en 'total_casos_satisfactorios.txt' en la misma carpeta de resultados.")

In [ ]:
#@siva compara los modulos de la integral absoluta de helicidad
import os
import numpy as np
import glob
import pandas as pd

# Definir la lista de triadas
triads = [
    "FRT", "PLB", "FLT", "PRB", "FRB",
    "PLT", "FLB", "PRT", "RTB", "FLP",
    "LTB", "FRP"
]

# Función para calcular módulos y sumar utilizando el diccionario `archivos_validos` con fechas
def calcular_modulos_y_sumar_con_fechas(archivos_validos, root_folder):
    resultados_por_fecha = {}  # Diccionario para almacenar los resultados por fecha
    total_casos_satisfactorios = 0  # Contador total de casos satisfactorios

    for lab in archivos_validos.keys():
        for turno in archivos_validos[lab].keys():
            for triad in triads:
                experimental_archivos = archivos_validos[lab][turno]['experimental']
                base_archivos = archivos_validos[lab][turno]['base']
                
                # Crear un diccionario para acceder a los archivos base por número
                base_dict = {archivo['Número']: archivo['Ruta'] for archivo in base_archivos}

                for exp_archivo in experimental_archivos:
                    exp_num = exp_archivo['Número']
                    exp_fecha = exp_archivo['Fecha']  # Obtener la fecha del archivo experimental
                    exp_ruta_pattern = os.path.join(root_folder, f"{exp_fecha} - {lab}", f"{exp_fecha}.{turno} - {lab}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Experimental_Color", triad, f"sum_absolute_helicity_{triad}_velocity_0{exp_num}med*.txt")
                    
                    # Usar glob para buscar el archivo experimental
                    exp_archivos_encontrados = glob.glob(exp_ruta_pattern)

                    if not exp_archivos_encontrados:
                        continue
                    
                    exp_ruta = exp_archivos_encontrados[0]  # Toma el primer archivo encontrado
                    exp_nombre = os.path.basename(exp_ruta)  # Obtener el nombre del archivo sin la ruta

                    # Calcular la suma de módulos para el archivo experimental
                    mod_exp_suma = sumar_modulos(exp_ruta)

                    # Inicializar el diccionario para almacenar los resultados por fecha si no existe
                    if exp_fecha not in resultados_por_fecha:
                        resultados_por_fecha[exp_fecha] = {triad: [] for triad in triads}

                    # Verificar si hay un archivo base con el mismo número y el siguiente
                    for num in (exp_num, exp_num + 1):
                        if num in base_dict:
                            base_ruta_pattern = os.path.join(root_folder, f"{exp_fecha} - {lab}", f"{exp_fecha}.{turno} - {lab}", "Data Analysis", "Processing Data", "Sum_Absolute_Helicity_Baseline", triad, f"sum_absolute_helicity_{triad}_velocity_0{num}med*.txt")
                            
                            # Usar glob para buscar el archivo base
                            base_archivos_encontrados = glob.glob(base_ruta_pattern)

                            if not base_archivos_encontrados:
                                print(f"No se encontró archivo base para el patrón: {base_ruta_pattern}")
                                continue

                            base_ruta = base_archivos_encontrados[0]  # Toma el primer archivo base encontrado
                            base_nombre = os.path.basename(base_ruta)  # Obtener el nombre del archivo base sin la ruta
                            
                            # Calcular la suma de módulos para el archivo base
                            mod_base_suma = sumar_modulos(base_ruta)
                            dif_porcentual = (abs(mod_exp_suma - mod_base_suma) / mod_base_suma) * 100

                            # Determinar el resultado basado en la diferencia porcentual
                            if dif_porcentual < 10:
                                resultado = "No hay evidencia de intervención."
                            elif 10 <= dif_porcentual <= 33.9:
                                resultado = "Resultados inconclusos."
                            else:  # valor > 33.9
                                resultado = "Evidencia de intervención satisfactoria."
                                total_casos_satisfactorios += 1  # Aumentar contador total de casos satisfactorios

                            # Almacenar el resultado en la lista de resultados para la triada y fecha
                            resultados_por_fecha[exp_fecha][triad].append({
                                'Laboratorio': lab,
                                'Turno': turno,
                                'Archivo_Experimental': exp_nombre,
                                'Archivo_Base': base_nombre,
                                'Modulo_Experimental': mod_exp_suma,
                                'Modulo_Base': mod_base_suma,
                                'Diferencia_Porcentual': dif_porcentual,
                                'Resultado': resultado
                            })

    return resultados_por_fecha, total_casos_satisfactorios

# Función para calcular la suma de módulos de los vectores en un archivo
def sumar_modulos(filepath):
    suma_modulos = 0

    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                vector = parts[1].strip('()').split(',')
                valores = np.array([float(num.strip()) for num in vector])
                modulo = np.linalg.norm(valores)  # Calcular el módulo del vector
                suma_modulos += modulo

    return suma_modulos

# Función para guardar los resultados en un archivo Excel separado por cada fecha dentro de `Resultados_Triadas` en `root_folder`
def guardar_resultados_por_fecha(resultados_por_fecha, root_folder):
    # Crear la carpeta 'Resultados_Triadas_Helicidad' dentro de `root_folder` si no existe
    carpeta_resultados = os.path.join(root_folder, "Excels", 'Absolute Helicity Analysis')
    os.makedirs(carpeta_resultados, exist_ok=True)

    for fecha, resultados_triadas in resultados_por_fecha.items():
        # Crear un archivo Excel con hojas por triada para cada fecha
        nombre_archivo = f'resultados_triadas_{fecha}.xlsx'
        ruta_archivo = os.path.join(carpeta_resultados, nombre_archivo)

        with pd.ExcelWriter(ruta_archivo, engine='openpyxl') as writer:
            for triad, resultados_triadas_data in resultados_triadas.items():
                # Crear un DataFrame para cada triada y guardarlo en una hoja
                df = pd.DataFrame(resultados_triadas_data)
                if not df.empty:
                    # Ordenar columnas según el código anterior
                    df = df[['Laboratorio', 'Turno', 'Archivo_Experimental', 'Archivo_Base', 'Modulo_Experimental', 'Modulo_Base', 'Diferencia_Porcentual', 'Resultado']]
                df.to_excel(writer, sheet_name=triad, index=False)

        print(f"Resultados guardados en {ruta_archivo}")

    return carpeta_resultados

# Función para guardar el total de casos satisfactorios en un archivo .txt
def guardar_casos_satisfactorios_txt(total_casos_satisfactorios, carpeta_resultados):
    # Definir la ruta del archivo .txt para guardar el total de casos satisfactorios en la carpeta de resultados Excel
    ruta_txt = os.path.join(carpeta_resultados, 'total_casos_satisfactorios.txt')

    with open(ruta_txt, 'w') as file:
        file.write(f'Total de casos satisfactorios: {total_casos_satisfactorios}\n')
        print(f"Total de casos satisfactorios guardado en {ruta_txt}")

# Definir la ruta `root_folder` según tu entorno
# root_folder = 'E:/Filter Data Agos-Sep'  # Reemplaza esta línea con tu `root_folder`

# Llamada a la función para calcular módulos y sumar utilizando `archivos_validos`
resultados_por_fecha, total_casos_satisfactorios = calcular_modulos_y_sumar_con_fechas(archivos_validos, root_folder)

# Guardar los resultados en archivos Excel separados por fecha dentro de la carpeta `root_folder/Resultados_Triadas_Helicidad`
carpeta_resultados = guardar_resultados_por_fecha(resultados_por_fecha, root_folder)

# Guardar el total de casos satisfactorios en un archivo .txt en la misma carpeta donde están los Excel
guardar_casos_satisfactorios_txt(total_casos_satisfactorios, carpeta_resultados)

print("Resultados guardados por fecha en archivos Excel separados dentro de 'Resultados_Triadas_Helicidad_Absoluta'.")
print("Total de casos satisfactorios guardado en 'total_casos_satisfactorios.txt' en la misma carpeta de resultados.")

In [ ]:
#siva este codigo compara los modulos de la primera integral de enstrofia
import os
import numpy as np
import glob
import pandas as pd

# Definir la lista de triadas
triads = [
    "FRT", "PLB", "FLT", "PRB", "FRB",
    "PLT", "FLB", "PRT", "RTB", "FLP",
    "LTB", "FRP"
]

# Función para calcular módulos y sumar utilizando el diccionario `archivos_validos` con fechas
def calcular_modulos_y_sumar_con_fechas(archivos_validos, root_folder):
    resultados_por_fecha = {}  # Diccionario para almacenar los resultados por fecha
    total_casos_satisfactorios = 0  # Contador total de casos satisfactorios

    for lab in archivos_validos.keys():
        for turno in archivos_validos[lab].keys():
            for triad in triads:
                experimental_archivos = archivos_validos[lab][turno]['experimental']
                base_archivos = archivos_validos[lab][turno]['base']
                
                # Crear un diccionario para acceder a los archivos base por número
                base_dict = {archivo['Número']: archivo['Ruta'] for archivo in base_archivos}

                for exp_archivo in experimental_archivos:
                    exp_num = exp_archivo['Número']
                    exp_fecha = exp_archivo['Fecha']  # Obtener la fecha del archivo experimental
                    exp_ruta_pattern = os.path.join(root_folder, f"{exp_fecha} - {lab}", f"{exp_fecha}.{turno} - {lab}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Experimental_Color", triad, f"sum_enstrophy_vorticity_{triad}_velocity_0{exp_num}med*.txt")
                    
                    # Usar glob para buscar el archivo experimental
                    exp_archivos_encontrados = glob.glob(exp_ruta_pattern)

                    if not exp_archivos_encontrados:
                        continue
                    
                    exp_ruta = exp_archivos_encontrados[0]  # Toma el primer archivo encontrado
                    exp_nombre = os.path.basename(exp_ruta)  # Obtener el nombre del archivo sin la ruta

                    # Calcular la suma de módulos para el archivo experimental
                    mod_exp_suma = sumar_modulos(exp_ruta)

                    # Inicializar el diccionario para almacenar los resultados por fecha si no existe
                    if exp_fecha not in resultados_por_fecha:
                        resultados_por_fecha[exp_fecha] = {triad: [] for triad in triads}

                    # Verificar si hay un archivo base con el mismo número y el siguiente
                    for num in (exp_num, exp_num + 1):
                        if num in base_dict:
                            base_ruta_pattern = os.path.join(root_folder, f"{exp_fecha} - {lab}", f"{exp_fecha}.{turno} - {lab}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Baseline", triad, f"sum_enstrophy_vorticity_{triad}_velocity_0{num}med*.txt")
                            
                            # Usar glob para buscar el archivo base
                            base_archivos_encontrados = glob.glob(base_ruta_pattern)

                            if not base_archivos_encontrados:
                                print(f"No se encontró archivo base para el patrón: {base_ruta_pattern}")
                                continue

                            base_ruta = base_archivos_encontrados[0]  # Toma el primer archivo base encontrado
                            base_nombre = os.path.basename(base_ruta)  # Obtener el nombre del archivo base sin la ruta
                            
                            # Calcular la suma de módulos para el archivo base
                            mod_base_suma = sumar_modulos(base_ruta)
                            dif_porcentual = (abs(mod_exp_suma - mod_base_suma) / mod_base_suma) * 100

                            # Determinar el resultado basado en la diferencia porcentual
                            if dif_porcentual < 10:
                                resultado = "No hay evidencia de intervención."
                            elif 10 <= dif_porcentual <= 33.9:
                                resultado = "Resultados inconclusos."
                            else:  # valor > 33.9
                                resultado = "Evidencia de intervención satisfactoria."
                                total_casos_satisfactorios += 1  # Aumentar contador total de casos satisfactorios

                            # Almacenar el resultado en la lista de resultados para la triada y fecha
                            resultados_por_fecha[exp_fecha][triad].append({
                                'Laboratorio': lab,
                                'Turno': turno,
                                'Archivo_Experimental': exp_nombre,
                                'Archivo_Base': base_nombre,
                                'Modulo_Experimental': mod_exp_suma,
                                'Modulo_Base': mod_base_suma,
                                'Diferencia_Porcentual': dif_porcentual,
                                'Resultado': resultado
                            })

    return resultados_por_fecha, total_casos_satisfactorios

# Función para calcular la suma de módulos de los vectores en un archivo
def sumar_modulos(filepath):
    suma_modulos = 0

    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                vector = parts[1].strip('()').split(',')
                valores = np.array([float(num.strip()) for num in vector])
                modulo = np.linalg.norm(valores)  # Calcular el módulo del vector
                suma_modulos += modulo

    return suma_modulos

# Función para guardar los resultados en un archivo Excel separado por cada fecha dentro de `Resultados_Triadas` en `root_folder`
def guardar_resultados_por_fecha(resultados_por_fecha, root_folder):
    # Crear la carpeta 'Resultados_Triadas_Helicidad' dentro de `root_folder` si no existe
    carpeta_resultados = os.path.join(root_folder, "Excels", 'Enstrophy Analysis')
    os.makedirs(carpeta_resultados, exist_ok=True)

    for fecha, resultados_triadas in resultados_por_fecha.items():
        # Crear un archivo Excel con hojas por triada para cada fecha
        nombre_archivo = f'resultados_triadas_{fecha}.xlsx'
        ruta_archivo = os.path.join(carpeta_resultados, nombre_archivo)

        with pd.ExcelWriter(ruta_archivo, engine='openpyxl') as writer:
            for triad, resultados_triadas_data in resultados_triadas.items():
                # Crear un DataFrame para cada triada y guardarlo en una hoja
                df = pd.DataFrame(resultados_triadas_data)
                if not df.empty:
                    # Ordenar columnas según el código anterior
                    df = df[['Laboratorio', 'Turno', 'Archivo_Experimental', 'Archivo_Base', 'Modulo_Experimental', 'Modulo_Base', 'Diferencia_Porcentual', 'Resultado']]
                df.to_excel(writer, sheet_name=triad, index=False)

        print(f"Resultados guardados en {ruta_archivo}")

    return carpeta_resultados

# Función para guardar el total de casos satisfactorios en un archivo .txt
def guardar_casos_satisfactorios_txt(total_casos_satisfactorios, carpeta_resultados):
    # Definir la ruta del archivo .txt para guardar el total de casos satisfactorios en la carpeta de resultados Excel
    ruta_txt = os.path.join(carpeta_resultados, 'total_casos_satisfactorios.txt')

    with open(ruta_txt, 'w') as file:
        file.write(f'Total de casos satisfactorios: {total_casos_satisfactorios}\n')
        print(f"Total de casos satisfactorios guardado en {ruta_txt}")

# Definir la ruta `root_folder` según tu entorno
# root_folder = 'E:/Filter Data Agos-Sep'  # Reemplaza esta línea con tu `root_folder`

# Llamada a la función para calcular módulos y sumar utilizando `archivos_validos`
resultados_por_fecha, total_casos_satisfactorios = calcular_modulos_y_sumar_con_fechas(archivos_validos, root_folder)

# Guardar los resultados en archivos Excel separados por fecha dentro de la carpeta `root_folder/Resultados_Triadas_Helicidad`
carpeta_resultados = guardar_resultados_por_fecha(resultados_por_fecha, root_folder)

# Guardar el total de casos satisfactorios en un archivo .txt en la misma carpeta donde están los Excel
guardar_casos_satisfactorios_txt(total_casos_satisfactorios, carpeta_resultados)

print("Resultados guardados por fecha en archivos Excel separados dentro de 'Resultados_Triadas_Enstrofia'.")
print("Total de casos satisfactorios guardado en 'total_casos_satisfactorios.txt' en la misma carpeta de resultados.")

In [33]:
import os
import numpy as np
import glob
import pandas as pd

# Definir la lista de triadas
triads = [
    "FRT", "PLB", "FLT", "PRB", "FRB",
    "PLT", "FLB", "PRT", "RTB", "FLP",
    "LTB", "FRP"
]

# Lista de colores
colores = ["AMARILLA", "ROJA", "MORADA", "VERDE", "AZUL", "NARANJA"]

# Función para calcular módulos y sumar utilizando el diccionario `archivos_validos` con fechas
def calcular_modulos_y_sumar_con_fechas(archivos_validos, root_folder):
    resultados_por_fecha = {}  # Diccionario para almacenar los resultados por fecha
    total_casos_satisfactorios = 0  # Contador total de casos satisfactorios

    for lab in archivos_validos.keys():
        for turno in archivos_validos[lab].keys():
            for triad in triads:
                experimental_archivos = archivos_validos[lab][turno]['experimental']
                base_archivos = archivos_validos[lab][turno]['base']

                # Crear un diccionario para acceder a los archivos base por número
                base_dict = {archivo['Número']: archivo['Ruta'] for archivo in base_archivos}

                for exp_archivo in experimental_archivos:
                    exp_num = exp_archivo['Número']
                    exp_color = exp_archivo['Color']  # Obtener el color del archivo experimental
                    exp_fecha = exp_archivo['Fecha']  # Obtener la fecha del archivo experimental
                    exp_ruta_pattern = os.path.join(root_folder, f"{exp_fecha} - {lab}", f"{exp_fecha}.{turno} - {lab}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Experimental_Color", triad, f"sum_enstrophy_vorticity_{triad}_velocity_0{exp_num}med{exp_color}.txt")

                    # Usar glob para buscar el archivo experimental
                    exp_archivos_encontrados = glob.glob(exp_ruta_pattern)

                    if not exp_archivos_encontrados:
                        continue

                    exp_ruta = exp_archivos_encontrados[0]  # Toma el primer archivo encontrado
                    exp_nombre = os.path.basename(exp_ruta)  # Obtener el nombre del archivo sin la ruta

                    # Calcular la suma de módulos para el archivo experimental
                    mod_exp_suma = sumar_modulos(exp_ruta)

                    # Inicializar el diccionario para almacenar los resultados por fecha si no existe
                    if exp_fecha not in resultados_por_fecha:
                        resultados_por_fecha[exp_fecha] = {triad: [] for triad in triads}

                    # Verificar si hay un archivo base con el mismo número y el siguiente
                    for num in (exp_num, exp_num + 1):
                        if num in base_dict:
                            base_ruta_pattern = os.path.join(root_folder, f"{exp_fecha} - {lab}", f"{exp_fecha}.{turno} - {lab}", "Data Analysis", "Processing Data", "Sum_Enstrophy_Baseline", triad, f"sum_enstrophy_vorticity_{triad}_velocity_0{num}med*.txt")

                            # Usar glob para buscar el archivo base
                            base_archivos_encontrados = glob.glob(base_ruta_pattern)

                            if not base_archivos_encontrados:
                                print(f"No se encontró archivo base para el patrón: {base_ruta_pattern}")
                                continue

                            base_ruta = base_archivos_encontrados[0]  # Toma el primer archivo base encontrado
                            base_nombre = os.path.basename(base_ruta)  # Obtener el nombre del archivo base sin la ruta

                            # Calcular la suma de módulos para el archivo base
                            mod_base_suma = sumar_modulos(base_ruta)
                            dif_porcentual = (abs(mod_exp_suma - mod_base_suma) / mod_base_suma) * 100

                            # Determinar el resultado basado en la diferencia porcentual
                            if dif_porcentual < 10:
                                resultado = "No hay evidencia de intervención."
                            elif 10 <= dif_porcentual <= 33.9:
                                resultado = "Resultados inconclusos."
                            else:  # valor > 33.9
                                resultado = "Evidencia de intervención satisfactoria."
                                total_casos_satisfactorios += 1  # Aumentar contador total de casos satisfactorios

                            # Almacenar el resultado en la lista de resultados para la triada y fecha
                            resultados_por_fecha[exp_fecha][triad].append({
                                'Laboratorio': lab,
                                'Turno': turno,
                                'Archivo_Experimental': exp_nombre,
                                'Archivo_Base': base_nombre,
                                'Modulo_Experimental': mod_exp_suma,
                                'Modulo_Base': mod_base_suma,
                                'Diferencia_Porcentual': dif_porcentual,
                                'Resultado': resultado
                            })

    return resultados_por_fecha, total_casos_satisfactorios

# Función para calcular la suma de módulos de los vectores en un archivo
def sumar_modulos(filepath):
    suma_modulos = 0

    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split(' -> ')
            if len(parts) > 1:
                vector = parts[1].strip('()').split(',')
                valores = np.array([float(num.strip()) for num in vector])
                modulo = np.linalg.norm(valores)  # Calcular el módulo del vector
                suma_modulos += modulo

    return suma_modulos

# Función para guardar los resultados como PDFs separados por experimento y color
def guardar_resultados_como_pdfs(resultados_por_fecha, root_folder):
    carpeta_pdfs = os.path.join(root_folder, "PDFs")
    os.makedirs(carpeta_pdfs, exist_ok=True)

    for fecha, resultados_triadas in resultados_por_fecha.items():
        for triad, resultados_triadas_data in resultados_triadas.items():
            for resultado in resultados_triadas_data:
                archivo_nombre = f"{resultado['Archivo_Experimental'].split('_')[2]}_{resultado['Archivo_Experimental'].split('_')[4]}.pdf"
                archivo_ruta = os.path.join(carpeta_pdfs, archivo_nombre)

                # Generar el PDF (ejemplo simple con pandas)
                df = pd.DataFrame([resultado])
                df.to_csv(archivo_ruta.replace('.pdf', '.csv'), index=False)

                print(f"PDF guardado en: {archivo_ruta}")


In [ ]:
import os
import re
import re

def extraer_fecha(ruta):
    # Definir el patrón para coincidir con el formato de fecha ddMmmYY (como 12Ago24)
    patron_fecha = r"\d{2}[A-Za-z]{3}\d{2}"
    
    # Buscar la fecha en la cadena
    coincidencia = re.search(patron_fecha, ruta)
    
    if coincidencia:
        return coincidencia.group()  # Devolver la fecha encontrada
    else:
        return None  # Si no se encuentra, devolver None
    
# Función para decodificar el texto hexadecimal
def decodificar_hex(hex_string):
    try:
        hex_cleaned = hex_string.strip().replace(' ', '')
        decoded_string = bytes.fromhex(hex_cleaned).decode('ascii', errors='ignore').strip()
        return decoded_string
    except Exception as e:
        print(f"Error decodificando: {e}")
        return None

# Función para procesar el archivo y extraer la hora y el valor limpio
def procesar_archivo_para_hora_y_texto(ruta_archivo):
    resultados = []
    
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            match_hora = re.search(r'(\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}\.\d{3})', linea)
            if match_hora:
                hora = match_hora.group(1).split(' ')[1]  # Obtener solo la parte de la hora
                partes = linea.strip().split(',')
                if len(partes) > 3:
                    hex_data = partes[-1]
                    decoded_value = decodificar_hex(hex_data)

                    if decoded_value:
                        resultados.append(f"{hora}, {decoded_value}")
                    else:
                        resultados.append(f"{hora}, .")
    
    return resultados

# Función para guardar los resultados en un archivo de salida
def guardar_resultados(ruta_salida, resultados):
    with open(ruta_salida, 'w') as archivo_salida:
        for resultado in resultados:
            archivo_salida.write(resultado + '\n')

# Función para construir la ruta del archivo a procesar
def construir_ruta(root_folder, fecha, lab, turno, color, numero):
    colorin = color.lower()
    return os.path.join(root_folder, f"{fecha} - {lab}", f"{fecha}.{turno} - {lab}", "Pesajes", f"{numero:02d}med{colorin}.log")

# Función para filtrar resultados
def filtrar_resultados(resultados):
    # Expresión regular para el formato deseado
    pattern = r'^\d{2}:\d{2}:\d{2}\.\d{3}, \d+\.\d{2}g$'
    return [resultado for resultado in resultados if re.match(pattern, resultado.strip())]

# Si este es el archivo principal
if __name__ == "__main__":
    fecha = extraer_fecha(root_folder)  # Asegúrate de definir esta función
    turnos = ["M", "V"]
    colores = ["Amarilla", "Roja", "Azul", "Verde", "Morada", "Negra", "Naranja"]
    numeros_archivo = range(1, 11)

    for lab in labs:
        for turno in turnos:
            for color in colores:
                for numero in numeros_archivo:
                    ruta_archivo = construir_ruta(root_folder, fecha, lab, turno, color, numero)
                    #print(ruta_archivo)
                    
                    colorin = color.lower()
                    if os.path.exists(ruta_archivo):
                        ruta_salida = os.path.join(os.path.dirname(ruta_archivo), f"{numero:02d}med{colorin}.txt")

                        print(f"\nProcesando para Lab: {lab}, Turno: {turno}, Color: {color}, Archivo: {numero:02d}")
                        print(f"Archivo a procesar: {ruta_archivo}")
                        
                        # Procesar el archivo para extraer hora y texto limpio
                        resultados = procesar_archivo_para_hora_y_texto(ruta_archivo)

                        # Filtrar resultados
                        resultados_filtrados = filtrar_resultados(resultados)

                        # Guardar los resultados filtrados en el archivo de salida
                        guardar_resultados(ruta_salida, resultados_filtrados)
                        print(f"Archivo encontrado: {ruta_archivo}")
                    #else:
                     #   print(f"Archivo no encontrado: {ruta_archivo}")
    print("\nProcesamiento completo.")

In [ ]:
import os
import re
import pandas as pd

# Función para extraer la fecha del nombre del directorio raíz
def extraer_fecha(root_folder):
    nombre_directorio = os.path.basename(root_folder)
    match = re.search(r'(\d{2}[A-Za-z]{3}\d{2})', nombre_directorio)
    
    if match:
        fecha_str = match.group(1)  # Captura la fecha en formato '16Sep24'
        return fecha_str
    else:
        raise ValueError(f"No se encontró una fecha válida en el nombre del directorio {root_folder}.")

# Función para cargar y procesar los datos
def cargar_y_procesar_datos(ruta_archivo):
    datos_procesados = []
    
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            partes = linea.split(',')
            hora = partes[0].strip()  # Extrae la hora
            valor = partes[1].strip()  # Extrae el valor con 'g'
            
            # Elimina la 'g' y convierte a float
            valor_numero = float(valor.replace('g', ''))
            # Aplica el tratamiento Dato * 9.81 / 5.00
            valor_procesado = valor_numero * 9.81 / 5.00
            
            # Guarda la hora y el valor procesado
            datos_procesados.append(f"{hora}, {valor_procesado:.2f}\n")
    
    return datos_procesados

# Función para guardar los datos procesados en un archivo .txt
def guardar_datos_txt(datos_procesados, ruta_salida):
    with open(ruta_salida, 'w') as archivo_salida:
        archivo_salida.writelines(datos_procesados)
    print(f"Datos procesados guardados en: {ruta_salida}")

# Función principal para procesar y guardar archivos
def procesar_archivos(root_folder):
    turnos = ["M", "V"]
    colores = ["Amarilla", "Roja", "Azul", "Verde", "Morada", "Negra", "Naranja"]
    numeros_archivo = range(1, 11)

    fecha = extraer_fecha(root_folder)

    for lab in labs:
        for turno in turnos:
            for color in colores:
                for numero in numeros_archivo:
                    colorin = color.lower()
                    ruta_archivo = os.path.join(root_folder, f"{fecha} - {lab}", f"{fecha}.{turno} - {lab}", "Pesajes", f"0{numero}med{colorin}.txt")
                    
                    if os.path.exists(ruta_archivo):
                        # Cargar y procesar los datos del archivo
                        datos_procesados = cargar_y_procesar_datos(ruta_archivo)

                        # Definir la carpeta y el nombre del archivo de salida
                        output_folder = os.path.join(root_folder, f"{fecha} - {lab}", f"{fecha}.{turno} - {lab}", "Pesajes", "procesados")
                        os.makedirs(output_folder, exist_ok=True)
                        ruta_salida = os.path.join(output_folder, f"resultados_0{numero}_{colorin}.txt")

                        print(ruta_salida)

                        # Guardar los datos procesados en un archivo .txt
                        guardar_datos_txt(datos_procesados, ruta_salida)
                    #else:
                        print(f"Archivo no encontrado: {ruta_archivo}")

# Si este es el archivo principal
if __name__ == "__main__":
    procesar_archivos(root_folder)
    print("\nProcesamiento completo.")

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Definir un diccionario para mapear los nombres de los colores a los colores de matplotlib
color_map = {
    "amarilla": "yellow",
    "roja": "red",
    "azul": "blue",
    "verde": "green",
    "morada": "purple",
    "negra": "black",
    "naranja": "orange"
}

# Función para cargar datos y crear DataFrame
def cargar_datos(ruta_archivo):
    # Leer archivo en un DataFrame
    df = pd.read_csv(ruta_archivo, header=None, names=["Timestamp", "Valor"])
    
    # Convertir el timestamp a formato de tiempo de pandas para facilitar el graficado
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S.%f')
    
    return df

# Función para graficar serie de tiempo como scatter plot con color especificado
def graficar_serie_tiempo(df, titulo, ruta_salida, color):
    plt.figure(figsize=(10, 6))
    plt.scatter(df['Timestamp'], df['Valor'], color=color)
    plt.xlabel('Time')
    plt.ylabel('Gravity (m/s^2)')
    plt.title(titulo)
    plt.grid()
    plt.xticks(rotation=45)
    
    # Guardar la gráfica en la ruta especificada
    plt.savefig(ruta_salida)
    plt.close()  # Cierra la gráfica para no mostrarla en la terminal
    print(f"Gráfica guardada en: {ruta_salida}")

# Función principal para procesar y graficar
def procesar_y_graficar_archivos(root_folder):
    fecha = extraer_fecha(root_folder)  # Usar la función extraer_fecha definida previamente
    turnos = ["M", "V"]
    #colores = ["Amarilla", "Roja", "Azul", "Verde", "Morada", "Negra"]
    numeros_archivo = range(1, 11)

    for lab in labs:
        for turno in turnos:
            for color in colores:
                for numero in numeros_archivo:
                    colorin = color.lower()
                    ruta_archivo = os.path.join(root_folder, f"{fecha} - {lab}", f"{fecha}.{turno} - {lab}", "Pesajes", "procesados", f"resultados_0{numero}_{colorin}.txt")
                    
                    if os.path.exists(ruta_archivo):
                        # Cargar los datos del archivo procesado
                        df = cargar_datos(ruta_archivo)
                        
                        # Crear el título de la gráfica
                        titulo = f"Time Series - Lab {lab} - Shift {turno} - {color} - Measure {numero}"
                        
                        # Definir la carpeta y el nombre del archivo de salida para la gráfica
                        output_folder = os.path.join(root_folder, f"{fecha} - {lab}", f"{fecha}.{turno} - {lab}", "Pesajes", "graficas")
                        os.makedirs(output_folder, exist_ok=True)
                        ruta_salida_grafica = os.path.join(output_folder, f"graph_0{numero}_{colorin}.png")
                        
                        # Obtener el color correspondiente a partir del nombre del archivo
                        color_grafica = color_map.get(colorin, 'blue')  # Default a 'blue' si no se encuentra el color
                        
                        # Graficar la serie de tiempo como scatter plot con el color correspondiente
                        graficar_serie_tiempo(df, titulo, ruta_salida_grafica, color_grafica)
                    else:
                        print(f"Archivo no encontrado: {ruta_archivo}")

# Ejecutar el procesamiento y graficado
if __name__ == "__main__":
    procesar_y_graficar_archivos(root_folder)
    print("\nProcesamiento y graficado completo.")

***

import os
import re
import shutil

# Función para crear una copia de respaldo del archivo
def backup_raw_file(filepath, root_folder, backup_folder):
    relative_path = os.path.relpath(filepath, root_folder)
    backup_path = os.path.join(backup_folder, relative_path)
    os.makedirs(os.path.dirname(backup_path), exist_ok=True)
    shutil.copy(filepath, backup_path)
    print(f"Archivo respaldado en: {backup_path}")

# Función para extraer estampas de tiempo
def extract_timestamps(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    if lines:
        first_line = re.match(r'^\d{2}:\d{2}:\d{2}\.\d{3}', lines[0])
        last_line = re.match(r'^\d{2}:\d{2}:\d{2}\.\d{3}', lines[-1])
        first_time = first_line.group(0) if first_line else "NA"
        last_time = last_line.group(0) if last_line else "NA"
        return first_time, last_time
    return "NA", "NA"

# Función para actualizar el formato de las líneas y registrar en la bitácora
def update_file_format(filepath):
    updated_lines = []
    print(f"Actualizando formato del archivo: {filepath}")
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            if re.match(r'^\d{2}:\d{2}:\d{2}\.\d{3} -> \d{2}:\d{2}:\d{2}\.\d{3} -> ', line):
                new_line = re.sub(r'^\d{2}:\d{2}:\d{2}\.\d{3} -> ', '', line.strip())
                updated_lines.append(new_line)
            else:
                updated_lines.append(line.strip())

    with open(filepath, 'w', encoding='utf-8') as file:
        for line in updated_lines:
            file.write(line + '\n')
    print(f"Formato actualizado en: {filepath}")

# Función para escanear un directorio y procesar archivos con sufijos específicos
def scan_directory(root_folder, file_suffixes, backup_folder):
    for root, dirs, files in os.walk(root_folder):
        dirs[:] = [d for d in dirs if d not in ["Data Analysis", "Programas"]]  # Excluir carpetas
        for filename in files:
            if any(filename.endswith(suffix) for suffix in file_suffixes):
                file_path = os.path.join(root, filename)
                try:
                    backup_raw_file(file_path, root_folder, backup_folder)
                    update_file_format(file_path)
                except Exception as e:
                    print(f"Error procesando el archivo {file_path}: {e}")

# Configuración inicial

backup_folder = os.path.join(root_folder, "Backup Raw Data")
file_suffixes = [".txt"]

# Realiza la búsqueda y procesa los archivos
scan_directory(root_folder, file_suffixes, backup_folder)
